In [ ]:
# import pandas as pd

# df = pd.read_csv("data/chicago/TRACT_METADATA_2021.txt", sep='\t')
# df = df[df['Short_Name'].str.contains(r'^.{2}e|.{3}e|.{4}e')]
# df.to_csv("data/chicago/chicago_TRACT_METADATA_2021.csv", index=False)
# df

## Selecting variables

In [8]:
import fiona

fiona.listlayers('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb')

['X01_AGE_AND_SEX',
 'X02_RACE',
 'X03_HISPANIC_OR_LATINO_ORIGIN',
 'X04_ANCESTRY',
 'X05_FOREIGN_BORN_CITIZENSHIP',
 'X06_PLACE_OF_BIRTH',
 'X07_MIGRATION',
 'X08_COMMUTING',
 'X09_CHILDREN_HOUSEHOLD_RELATIONSHIP',
 'X10_GRANDPARENTS_GRANDCHILDREN',
 'X11_HOUSEHOLD_FAMILY_SUBFAMILIES',
 'X12_MARITAL_STATUS_AND_HISTORY',
 'X13_FERTILITY',
 'X14_SCHOOL_ENROLLMENT',
 'X15_EDUCATIONAL_ATTAINMENT',
 'X16_LANGUAGE_SPOKEN_AT_HOME',
 'X17_POVERTY',
 'X18_DISABILITY',
 'X19_INCOME',
 'X20_EARNINGS',
 'X21_VETERAN_STATUS',
 'X22_FOOD_STAMPS',
 'X23_EMPLOYMENT_STATUS',
 'X25_HOUSING_CHARACTERISTICS',
 'X27_HEALTH_INSURANCE',
 'X28_COMPUTER_AND_INTERNET_USE',
 'X29_VOTING_AGE_POPULATION',
 'X99_IMPUTATION',
 'X24_INDUSTRY_OCCUPATION',
 'X26_GROUP_QUARTERS',
 'TRACT_METADATA_2021',
 'ACS_2021_5YR_TRACT']

Metadata

In [9]:
import pandas as pd

meta = pd.read_csv("data/chicago/chicago_var.csv")
meta["Short_Name"]

0        B01001e1
1        B01001e2
2        B01001e3
3        B01001e4
4        B01001e5
          ...    
773     B25032Ie7
774     B25032Ie8
775     B25032Ie9
776    B25032Ie10
777    B25032Ie11
Name: Short_Name, Length: 778, dtype: object

Geometry

In [10]:
import geopandas as gpd

gdf_geo = gpd.read_file('data/chicago/chicago_ACS_2021_5YR_TRACT.shp')
gdf_geo = gdf_geo.rename(columns={'GEOID': 'GEOID_x'})
gdf_geo.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID_x,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area,GEOID_Data,geometry
0,17,031,221000,17031221000,2210,Census Tract 2210,G5020,S,220053.0,0.0,+41.9209604,-087.7152526,0.021157,0.000024,14000US17031221000,"POLYGON ((-87.71699 41.92460, -87.71675 41.924..."
1,17,031,221100,17031221100,2211,Census Tract 2211,G5020,S,440042.0,0.0,+41.9210205,-087.7103609,0.027709,0.000048,14000US17031221100,"POLYGON ((-87.71374 41.92464, -87.71366 41.924..."
2,17,031,242800,17031242800,2428,Census Tract 2428,G5020,S,439630.0,0.0,+41.8926320,-087.6903237,0.032738,0.000048,14000US17031242800,"POLYGON ((-87.69670 41.89565, -87.69646 41.895..."
3,17,031,242900,17031242900,2429,Census Tract 2429,G5020,S,323819.0,0.0,+41.8921955,-087.6842896,0.024218,0.000035,14000US17031242900,"POLYGON ((-87.68684 41.89574, -87.68635 41.895..."
4,17,031,243000,17031243000,2430,Census Tract 2430,G5020,S,324547.0,0.0,+41.8922753,-087.6793981,0.024178,0.000035,14000US17031243000,"POLYGON ((-87.68195 41.89583, -87.68146 41.895..."


Race & hispanic origin

In [23]:
## race
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B02001e")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X02_RACE', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['white'] = gdf['B02001e2'] / gdf['B02001e1']
gdf['black'] = gdf['B02001e3'] / gdf['B02001e1']
gdf['race_other'] = (gdf['B02001e1'] - gdf['B02001e2'] - gdf['B02001e3']) / gdf['B02001e1']
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B02001e")])
gdf_race = gdf_geo[['GEOID_Data']].merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_race = gdf_race.drop(columns=['GEOID', 'geometry'])

## hispanic origin
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B03001e")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X03_HISPANIC_OR_LATINO_ORIGIN', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['hisp'] = gdf['B03001e3'] / gdf['B03001e1']
gdf['nhisp'] = gdf['B03001e2'] / gdf['B03001e1']
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B03001e")])
gdf_race = gdf_race.merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_race = gdf_race.drop(columns=['GEOID', 'geometry'])

gdf_race.to_csv("data/chicago/var/race_ethinicity.csv", index=False)
gdf_race

,GEOID_Data,white,black,race_other,hisp,nhisp
0,14000US17031221000,0.661089,0.062646,0.276265,0.499222,0.500778
1,14000US17031221100,0.648510,0.100489,0.251000,0.360605,0.639395
2,14000US17031242800,0.881486,0.007687,0.110826,0.227418,0.772582
3,14000US17031242900,0.665062,0.052434,0.282504,0.231675,0.768325
4,14000US17031243000,0.627896,0.012721,0.359382,0.243071,0.756929
...,...,...,...,...,...,...
792,14000US17031440201,0.003885,0.971188,0.024927,0.000000,1.000000
793,14000US17031540101,0.035943,0.932304,0.031753,0.041896,0.958104
794,14000US17031050300,0.646098,0.005808,0.348094,0.177495,0.822505
795,14000US17031830700,0.484930,0.335212,0.179858,0.128645,0.871355


Age

In [28]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B01001e")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X01_AGE_AND_SEX', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['ag1'] = gdf[[f'B01001e{i}' for i in range(3, 7)]].sum(axis=1) + gdf[[f'B01001e{i}' for i in range(27, 31)]].sum(axis=1)
gdf['ag2'] = gdf[[f'B01001e{i}' for i in range(7, 11)]].sum(axis=1) + gdf[[f'B01001e{i}' for i in range(31, 35)]].sum(axis=1)
gdf['ag3'] = gdf[[f'B01001e{i}' for i in range(11, 15)]].sum(axis=1) + gdf[[f'B01001e{i}' for i in range(35, 39)]].sum(axis=1)
gdf['ag4'] = gdf[[f'B01001e{i}' for i in range(15, 20)]].sum(axis=1) + gdf[[f'B01001e{i}' for i in range(39, 44)]].sum(axis=1)
gdf['ag5'] = gdf[[f'B01001e{i}' for i in range(20, 26)]].sum(axis=1) + gdf[[f'B01001e{i}' for i in range(44, 50)]].sum(axis=1)
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B01001e")])
gdf_age = gdf_geo[['GEOID_Data']].merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_age = gdf_age.drop(columns=['GEOID', 'geometry'])

## white
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B01001A")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X01_AGE_AND_SEX', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['white_ag1'] = gdf[[f'B01001Ae{i}' for i in range(3, 7)]].sum(axis=1) + gdf[[f'B01001Ae{i}' for i in range(18, 22)]].sum(axis=1)
gdf['white_ag2'] = gdf[[f'B01001Ae{i}' for i in range(7, 9)]].sum(axis=1) + gdf[[f'B01001Ae{i}' for i in range(22, 24)]].sum(axis=1)
gdf['white_ag3'] = gdf[[f'B01001Ae{i}' for i in range(9, 12)]].sum(axis=1) + gdf[[f'B01001Ae{i}' for i in range(24, 27)]].sum(axis=1)
gdf['white_ag4'] = gdf[[f'B01001Ae{i}' for i in range(12, 14)]].sum(axis=1) + gdf[[f'B01001Ae{i}' for i in range(27, 29)]].sum(axis=1)
gdf['white_ag5'] = gdf[[f'B01001Ae{i}' for i in range(14, 17)]].sum(axis=1) + gdf[[f'B01001Ae{i}' for i in range(29, 32)]].sum(axis=1)
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B01001A")])
gdf_age = gdf_age.merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_age = gdf_age.drop(columns=['GEOID', 'geometry'])

## black
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B01001B")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X01_AGE_AND_SEX', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['black_ag1'] = gdf[[f'B01001Be{i}' for i in range(3, 7)]].sum(axis=1) + gdf[[f'B01001Be{i}' for i in range(18, 22)]].sum(axis=1)
gdf['black_ag2'] = gdf[[f'B01001Be{i}' for i in range(7, 9)]].sum(axis=1) + gdf[[f'B01001Be{i}' for i in range(22, 24)]].sum(axis=1)
gdf['black_ag3'] = gdf[[f'B01001Be{i}' for i in range(9, 12)]].sum(axis=1) + gdf[[f'B01001Be{i}' for i in range(24, 27)]].sum(axis=1)
gdf['black_ag4'] = gdf[[f'B01001Be{i}' for i in range(12, 14)]].sum(axis=1) + gdf[[f'B01001Be{i}' for i in range(27, 29)]].sum(axis=1)
gdf['black_ag5'] = gdf[[f'B01001Be{i}' for i in range(14, 17)]].sum(axis=1) + gdf[[f'B01001Be{i}' for i in range(29, 32)]].sum(axis=1)
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B01001B")])
gdf_age = gdf_age.merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_age = gdf_age.drop(columns=['GEOID', 'geometry'])

## other races
gdf_age['other_ag1'] = gdf_age['ag1'] - gdf_age['white_ag1'] - gdf_age['black_ag1']
gdf_age['other_ag2'] = gdf_age['ag2'] - gdf_age['white_ag2'] - gdf_age['black_ag2']
gdf_age['other_ag3'] = gdf_age['ag3'] - gdf_age['white_ag3'] - gdf_age['black_ag3']
gdf_age['other_ag4'] = gdf_age['ag4'] - gdf_age['white_ag4'] - gdf_age['black_ag4']
gdf_age['other_ag5'] = gdf_age['ag5'] - gdf_age['white_ag5'] - gdf_age['black_ag5']

## white & non-hispanic
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B01001H")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X01_AGE_AND_SEX', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['nhisp_white_ag1'] = gdf[[f'B01001He{i}' for i in range(3, 7)]].sum(axis=1) + gdf[[f'B01001He{i}' for i in range(18, 22)]].sum(axis=1)
gdf['nhisp_white_ag2'] = gdf[[f'B01001He{i}' for i in range(7, 9)]].sum(axis=1) + gdf[[f'B01001He{i}' for i in range(22, 24)]].sum(axis=1)
gdf['nhisp_white_ag3'] = gdf[[f'B01001He{i}' for i in range(9, 12)]].sum(axis=1) + gdf[[f'B01001He{i}' for i in range(24, 27)]].sum(axis=1)
gdf['nhisp_white_ag4'] = gdf[[f'B01001He{i}' for i in range(12, 14)]].sum(axis=1) + gdf[[f'B01001He{i}' for i in range(27, 29)]].sum(axis=1)
gdf['nhisp_white_ag5'] = gdf[[f'B01001He{i}' for i in range(14, 17)]].sum(axis=1) + gdf[[f'B01001He{i}' for i in range(29, 32)]].sum(axis=1)
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B01001H")])
gdf_age = gdf_age.merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_age = gdf_age.drop(columns=['GEOID', 'geometry'])

## hispanic
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B01001I")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X01_AGE_AND_SEX', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['hisp_ag1'] = gdf[[f'B01001Ie{i}' for i in range(3, 7)]].sum(axis=1) + gdf[[f'B01001Ie{i}' for i in range(18, 22)]].sum(axis=1)
gdf['hisp_ag2'] = gdf[[f'B01001Ie{i}' for i in range(7, 9)]].sum(axis=1) + gdf[[f'B01001Ie{i}' for i in range(22, 24)]].sum(axis=1)
gdf['hisp_ag3'] = gdf[[f'B01001Ie{i}' for i in range(9, 12)]].sum(axis=1) + gdf[[f'B01001Ie{i}' for i in range(24, 27)]].sum(axis=1)
gdf['hisp_ag4'] = gdf[[f'B01001Ie{i}' for i in range(12, 14)]].sum(axis=1) + gdf[[f'B01001Ie{i}' for i in range(27, 29)]].sum(axis=1)
gdf['hisp_ag5'] = gdf[[f'B01001Ie{i}' for i in range(14, 17)]].sum(axis=1) + gdf[[f'B01001Ie{i}' for i in range(29, 32)]].sum(axis=1)
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B01001I")])
gdf_age = gdf_age.merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_age = gdf_age.drop(columns=['GEOID', 'geometry'])

## non-hispanic
gdf_age['nhisp_ag1'] = gdf_age['ag1'] - gdf_age['hisp_ag1']
gdf_age['nhisp_ag2'] = gdf_age['ag2'] - gdf_age['hisp_ag2']
gdf_age['nhisp_ag3'] = gdf_age['ag3'] - gdf_age['hisp_ag3']
gdf_age['nhisp_ag4'] = gdf_age['ag4'] - gdf_age['hisp_ag4']
gdf_age['nhisp_ag5'] = gdf_age['ag5'] - gdf_age['hisp_ag5']

gdf_age

,GEOID_Data,ag1,ag2,ag3,ag4,ag5,white_ag1,white_ag2,white_ag3,white_ag4,...,hisp_ag1,hisp_ag2,hisp_ag3,hisp_ag4,hisp_ag5,nhisp_ag1,nhisp_ag2,nhisp_ag3,nhisp_ag4,nhisp_ag5
0,14000US17031221000,332.0,325.0,1249.0,567.0,97.0,225.0,148.0,1000.0,264.0,...,253.0,134.0,429.0,411.0,56.0,79.0,191.0,820.0,156.0,41.0
1,14000US17031221100,521.0,340.0,2410.0,539.0,688.0,268.0,148.0,1717.0,261.0,...,355.0,103.0,469.0,248.0,447.0,166.0,237.0,1941.0,291.0,241.0
2,14000US17031242800,271.0,52.0,754.0,315.0,169.0,217.0,52.0,685.0,288.0,...,70.0,0.0,111.0,106.0,68.0,201.0,52.0,643.0,209.0,101.0
3,14000US17031242900,345.0,77.0,896.0,470.0,81.0,188.0,44.0,654.0,298.0,...,96.0,53.0,80.0,178.0,26.0,249.0,24.0,816.0,292.0,55.0
4,14000US17031243000,230.0,87.0,1236.0,415.0,233.0,138.0,19.0,888.0,289.0,...,37.0,22.0,206.0,77.0,193.0,193.0,65.0,1030.0,338.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,1818.0,312.0,1585.0,1754.0,709.0,0.0,0.0,24.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1818.0,312.0,1585.0,1754.0,709.0
793,14000US17031540101,1757.0,717.0,1305.0,587.0,169.0,53.0,10.0,55.0,28.0,...,68.0,10.0,55.0,28.0,29.0,1689.0,707.0,1250.0,559.0,140.0
794,14000US17031050300,689.0,37.0,965.0,778.0,286.0,353.0,30.0,632.0,592.0,...,152.0,9.0,140.0,137.0,51.0,537.0,28.0,825.0,641.0,235.0
795,14000US17031830700,581.0,323.0,1648.0,1132.0,397.0,86.0,165.0,969.0,594.0,...,77.0,79.0,181.0,107.0,81.0,504.0,244.0,1467.0,1025.0,316.0


In [29]:
gdf_age.loc[:, 'ag1':'ag5'] = gdf_age.loc[:, 'ag1':'ag5'].div(gdf_age.loc[:, 'ag1':'ag5'].sum(axis=1), axis=0)
gdf_age.loc[:, 'white_ag1':'white_ag5'] = gdf_age.loc[:, 'white_ag1':'white_ag5'].div(gdf_age.loc[:, 'white_ag1':'white_ag5'].sum(axis=1), axis=0)
gdf_age.loc[:, 'black_ag1':'black_ag5'] = gdf_age.loc[:, 'black_ag1':'black_ag5'].div(gdf_age.loc[:, 'black_ag1':'black_ag5'].sum(axis=1), axis=0)
gdf_age.loc[:, 'other_ag1':'other_ag5'] = gdf_age.loc[:, 'other_ag1':'other_ag5'].div(gdf_age.loc[:, 'other_ag1':'other_ag5'].sum(axis=1), axis=0)
gdf_age.loc[:, 'nhisp_white_ag1':'nhisp_white_ag5'] = gdf_age.loc[:, 'nhisp_white_ag1':'nhisp_white_ag5'].div(gdf_age.loc[:, 'nhisp_white_ag1':'nhisp_white_ag5'].sum(axis=1), axis=0)
gdf_age.loc[:, 'hisp_ag1':'hisp_ag5'] = gdf_age.loc[:, 'hisp_ag1':'hisp_ag5'].div(gdf_age.loc[:, 'hisp_ag1':'hisp_ag5'].sum(axis=1), axis=0)
gdf_age.loc[:, 'nhisp_ag1':'nhisp_ag5'] = gdf_age.loc[:, 'nhisp_ag1':'nhisp_ag5'].div(gdf_age.loc[:, 'nhisp_ag1':'nhisp_ag5'].sum(axis=1), axis=0)
gdf_age.to_csv("data/chicago/var/age.csv", index=False)
gdf_age

,GEOID_Data,ag1,ag2,ag3,ag4,ag5,white_ag1,white_ag2,white_ag3,white_ag4,...,hisp_ag1,hisp_ag2,hisp_ag3,hisp_ag4,hisp_ag5,nhisp_ag1,nhisp_ag2,nhisp_ag3,nhisp_ag4,nhisp_ag5
0,14000US17031221000,0.129183,0.126459,0.485992,0.220623,0.037743,0.132431,0.087110,0.588582,0.155386,...,0.197194,0.104443,0.334373,0.320343,0.043648,0.061383,0.148407,0.637141,0.121212,0.031857
1,14000US17031221100,0.115829,0.075589,0.535794,0.119831,0.152957,0.091875,0.050737,0.588618,0.089475,...,0.218866,0.063502,0.289149,0.152898,0.275586,0.057719,0.082406,0.674896,0.101182,0.083797
2,14000US17031242800,0.173607,0.033312,0.483024,0.201794,0.108264,0.157703,0.037791,0.497820,0.209302,...,0.197183,0.000000,0.312676,0.298592,0.191549,0.166667,0.043118,0.533167,0.173300,0.083748
3,14000US17031242900,0.184591,0.041199,0.479401,0.251471,0.043339,0.151247,0.035398,0.526146,0.239743,...,0.221709,0.122402,0.184758,0.411085,0.060046,0.173398,0.016713,0.568245,0.203343,0.038301
4,14000US17031243000,0.104498,0.039527,0.561563,0.188551,0.105861,0.099855,0.013748,0.642547,0.209117,...,0.069159,0.041121,0.385047,0.143925,0.360748,0.115846,0.039016,0.618247,0.202881,0.024010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.294270,0.050502,0.256556,0.283911,0.114762,0.000000,0.000000,1.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.294270,0.050502,0.256556,0.283911,0.114762
793,14000US17031540101,0.387431,0.158104,0.287762,0.129438,0.037266,0.325153,0.061350,0.337423,0.171779,...,0.357895,0.052632,0.289474,0.147368,0.152632,0.388723,0.162716,0.287687,0.128654,0.032221
794,14000US17031050300,0.250091,0.013430,0.350272,0.282396,0.103811,0.198315,0.016854,0.355056,0.332584,...,0.310838,0.018405,0.286299,0.280164,0.104294,0.236981,0.012357,0.364078,0.282877,0.103707
795,14000US17031830700,0.142367,0.079147,0.403823,0.277383,0.097280,0.043456,0.083375,0.489641,0.300152,...,0.146667,0.150476,0.344762,0.203810,0.154286,0.141732,0.068616,0.412542,0.288245,0.088864


Education

In [22]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B15003") or value.startswith("C15002")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X15_EDUCATIONAL_ATTAINMENT', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['less_high_school'] = gdf[[f'B15003e{i}' for i in range(2, 17)]].sum(axis=1)
gdf['high_school'] = gdf[[f'B15003e{i}' for i in range(17, 19)]].sum(axis=1)
gdf['college'] = gdf[[f'B15003e{i}' for i in range(19, 22)]].sum(axis=1)
gdf['bachelor_higher'] = gdf[[f'B15003e{i}' for i in range(22, 26)]].sum(axis=1)

## white
gdf['white_less_high_school'] = gdf['C15002Ae3'] + gdf['C15002Ae8']
gdf['white_high_school'] = gdf['C15002Ae4'] + gdf['C15002Ae9']
gdf['white_college'] = gdf['C15002Ae5'] + gdf['C15002Ae10']
gdf['white_bachelor_higher'] = gdf['C15002Ae6'] + gdf['C15002Ae11']

## black
gdf['black_less_high_school'] = gdf['C15002Be3'] + gdf['C15002Be8']
gdf['black_high_school'] = gdf['C15002Be4'] + gdf['C15002Be9']
gdf['black_college'] = gdf['C15002Be5'] + gdf['C15002Be10']
gdf['black_bachelor_higher'] = gdf['C15002Be6'] + gdf['C15002Be11']
gdf_edu = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not (value.startswith("B15003") or value.startswith("C15002"))]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_edu = gdf_edu.drop(columns=['GEOID', 'geometry'])

## other races
gdf_edu['other_less_high_school'] = gdf_edu['less_high_school'] - gdf_edu['white_less_high_school'] - gdf_edu['black_less_high_school']
gdf_edu['other_high_school'] = gdf_edu['high_school'] - gdf_edu['white_high_school'] - gdf_edu['black_high_school']
gdf_edu['other_college'] = gdf_edu['college'] - gdf_edu['white_college'] - gdf_edu['black_college']
gdf_edu['other_bachelor_higher'] = gdf_edu['bachelor_higher'] - gdf_edu['white_bachelor_higher'] - gdf_edu['black_bachelor_higher']

## white & non-hispanic
gdf['nhisp_white_less_high_school'] = gdf['C15002He3'] + gdf['C15002He8']
gdf['nhisp_white_high_school'] = gdf['C15002He4'] + gdf['C15002He9']
gdf['nhisp_white_college'] = gdf['C15002He5'] + gdf['C15002He10']
gdf['nhisp_white_bachelor_higher'] = gdf['C15002He6'] + gdf['C15002He11']

## hispanic
gdf['hisp_less_high_school'] = gdf['C15002Ie3'] + gdf['C15002Ie8']
gdf['hisp_high_school'] = gdf['C15002Ie4'] + gdf['C15002Ie9']
gdf['hisp_college'] = gdf['C15002Ie5'] + gdf['C15002Ie10']
gdf['hisp_bachelor_higher'] = gdf['C15002Ie6'] + gdf['C15002Ie11']
gdf_edu = gdf_edu.merge(gdf[[value for value in gdf.columns if value=="GEOID" or value.startswith("nhisp") or value.startswith("hisp")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_edu = gdf_edu.drop(columns=['GEOID'])

## non-hispanic
gdf_edu['nhisp_less_high_school'] = gdf_edu['less_high_school'] - gdf_edu['hisp_less_high_school']
gdf_edu['nhisp_high_school'] = gdf_edu['high_school'] - gdf_edu['hisp_high_school']
gdf_edu['nhisp_college'] = gdf_edu['college'] - gdf_edu['hisp_college']
gdf_edu['nhisp_bachelor_higher'] = gdf_edu['bachelor_higher'] - gdf_edu['hisp_bachelor_higher']

gdf_edu

,GEOID_Data,less_high_school,high_school,college,bachelor_higher,white_less_high_school,white_high_school,white_college,white_bachelor_higher,black_less_high_school,...,nhisp_white_college,nhisp_white_bachelor_higher,hisp_less_high_school,hisp_high_school,hisp_college,hisp_bachelor_higher,nhisp_less_high_school,nhisp_high_school,nhisp_college,nhisp_bachelor_higher
0,14000US17031221000,307.0,509.0,360.0,737.0,205.0,199.0,316.0,606.0,0.0,...,151.0,570.0,195.0,450.0,178.0,73.0,112.0,59.0,182.0,664.0
1,14000US17031221100,348.0,825.0,503.0,1961.0,185.0,455.0,319.0,1542.0,22.0,...,225.0,1396.0,271.0,554.0,174.0,165.0,77.0,271.0,329.0,1796.0
2,14000US17031242800,89.0,189.0,204.0,756.0,32.0,174.0,172.0,729.0,0.0,...,113.0,693.0,62.0,96.0,85.0,42.0,27.0,93.0,119.0,714.0
3,14000US17031242900,87.0,75.0,243.0,1042.0,19.0,63.0,174.0,755.0,0.0,...,150.0,729.0,80.0,28.0,74.0,102.0,7.0,47.0,169.0,940.0
4,14000US17031243000,33.0,177.0,443.0,1231.0,8.0,83.0,196.0,938.0,10.0,...,154.0,927.0,15.0,96.0,269.0,96.0,18.0,81.0,174.0,1135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,520.0,1112.0,1326.0,1090.0,0.0,24.0,0.0,0.0,520.0,...,0.0,0.0,0.0,0.0,0.0,0.0,520.0,1112.0,1326.0,1090.0
793,14000US17031540101,187.0,776.0,997.0,101.0,25.0,43.0,32.0,0.0,147.0,...,9.0,0.0,17.0,72.0,23.0,0.0,170.0,704.0,974.0,101.0
794,14000US17031050300,23.0,258.0,367.0,1381.0,15.0,190.0,309.0,883.0,0.0,...,262.0,852.0,23.0,102.0,70.0,133.0,0.0,156.0,297.0,1248.0
795,14000US17031830700,232.0,467.0,913.0,1565.0,109.0,162.0,281.0,1176.0,42.0,...,246.0,1113.0,168.0,44.0,94.0,63.0,64.0,423.0,819.0,1502.0


In [23]:
gdf_edu.loc[:, 'less_high_school':'bachelor_higher'] = gdf_edu.loc[:, 'less_high_school':'bachelor_higher'].div(gdf_edu.loc[:, 'less_high_school':'bachelor_higher'].sum(axis=1), axis=0)
gdf_edu.loc[:, 'white_less_high_school':'white_bachelor_higher'] = gdf_edu.loc[:, 'white_less_high_school':'white_bachelor_higher'].div(gdf_edu.loc[:, 'white_less_high_school':'white_bachelor_higher'].sum(axis=1), axis=0)
gdf_edu.loc[:, 'black_less_high_school':'black_bachelor_higher'] = gdf_edu.loc[:, 'black_less_high_school':'black_bachelor_higher'].div(gdf_edu.loc[:, 'black_less_high_school':'black_bachelor_higher'].sum(axis=1), axis=0)
gdf_edu.loc[:, 'other_less_high_school':'other_bachelor_higher'] = gdf_edu.loc[:, 'other_less_high_school':'other_bachelor_higher'].div(gdf_edu.loc[:, 'other_less_high_school':'other_bachelor_higher'].sum(axis=1), axis=0)
gdf_edu.loc[:, 'nhisp_white_less_high_school':'nhisp_white_bachelor_higher'] = gdf_edu.loc[:, 'nhisp_white_less_high_school':'nhisp_white_bachelor_higher'].div(gdf_edu.loc[:, 'nhisp_white_less_high_school':'nhisp_white_bachelor_higher'].sum(axis=1), axis=0)
gdf_edu.loc[:, 'hisp_less_high_school':'hisp_bachelor_higher'] = gdf_edu.loc[:, 'hisp_less_high_school':'hisp_bachelor_higher'].div(gdf_edu.loc[:, 'hisp_less_high_school':'hisp_bachelor_higher'].sum(axis=1), axis=0)
gdf_edu.loc[:, 'nhisp_less_high_school':'nhisp_bachelor_higher'] = gdf_edu.loc[:, 'nhisp_less_high_school':'nhisp_bachelor_higher'].div(gdf_edu.loc[:, 'nhisp_less_high_school':'nhisp_bachelor_higher'].sum(axis=1), axis=0)
gdf_edu.to_csv("data/chicago/var/education.csv", index=False)
gdf_edu

,GEOID_Data,less_high_school,high_school,college,bachelor_higher,white_less_high_school,white_high_school,white_college,white_bachelor_higher,black_less_high_school,...,nhisp_white_college,nhisp_white_bachelor_higher,hisp_less_high_school,hisp_high_school,hisp_college,hisp_bachelor_higher,nhisp_less_high_school,nhisp_high_school,nhisp_college,nhisp_bachelor_higher
0,14000US17031221000,0.160481,0.266074,0.188186,0.385259,0.154600,0.150075,0.238311,0.457014,0.000000,...,0.171982,0.649203,0.217634,0.502232,0.198661,0.081473,0.110128,0.058014,0.178958,0.652901
1,14000US17031221100,0.095683,0.226835,0.138301,0.539181,0.073970,0.181927,0.127549,0.616553,0.058981,...,0.126904,0.787366,0.232818,0.475945,0.149485,0.141753,0.031136,0.109584,0.133037,0.726243
2,14000US17031242800,0.071890,0.152666,0.164782,0.610662,0.028907,0.157182,0.155375,0.658537,0.000000,...,0.122030,0.748380,0.217544,0.336842,0.298246,0.147368,0.028332,0.097587,0.124869,0.749213
3,14000US17031242900,0.060124,0.051831,0.167934,0.720111,0.018793,0.062315,0.172107,0.746785,0.000000,...,0.160772,0.781350,0.281690,0.098592,0.260563,0.359155,0.006019,0.040413,0.145314,0.808255
4,14000US17031243000,0.017516,0.093949,0.235138,0.653397,0.006531,0.067755,0.160000,0.765714,0.357143,...,0.132530,0.797762,0.031513,0.201681,0.565126,0.201681,0.012784,0.057528,0.123580,0.806108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.128458,0.274704,0.327569,0.269269,0.000000,1.000000,0.000000,0.000000,0.134367,...,0.000000,0.000000,NaN,NaN,NaN,NaN,0.128458,0.274704,0.327569,0.269269
793,14000US17031540101,0.090733,0.376516,0.483746,0.049005,0.250000,0.430000,0.320000,0.000000,0.076682,...,0.529412,0.000000,0.151786,0.642857,0.205357,0.000000,0.087224,0.361211,0.499743,0.051821
794,14000US17031050300,0.011336,0.127156,0.180877,0.680631,0.010737,0.136006,0.221188,0.632069,0.000000,...,0.211120,0.686543,0.070122,0.310976,0.213415,0.405488,0.000000,0.091711,0.174603,0.733686
795,14000US17031830700,0.073025,0.146994,0.287378,0.492603,0.063079,0.093750,0.162616,0.680556,0.045504,...,0.164439,0.743984,0.455285,0.119241,0.254743,0.170732,0.022792,0.150641,0.291667,0.534900


Household type

In [25]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B11001")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X11_HOUSEHOLD_FAMILY_SUBFAMILIES', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['married_couple_family'] = gdf['B11001e3']
gdf['family_other'] = gdf['B11001e4']
gdf['nonfamily'] = gdf['B11001e7']

## white
gdf['white_married_couple_family'] = gdf['B11001Ae3']
gdf['white_family_other'] = gdf['B11001Ae4']
gdf['white_nonfamily'] = gdf['B11001Ae7']

## black
gdf['black_married_couple_family'] = gdf['B11001Be3']
gdf['black_family_other'] = gdf['B11001Be4']
gdf['black_nonfamily'] = gdf['B11001Be7']
gdf_hhtype = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not value.startswith("B11001")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_hhtype = gdf_hhtype.drop(columns=['GEOID', 'geometry'])

## other races
gdf_hhtype['other_married_couple_family'] = gdf_hhtype['married_couple_family'] - gdf_hhtype['white_married_couple_family'] - gdf_hhtype['black_married_couple_family']
gdf_hhtype['other_family_other'] = gdf_hhtype['family_other'] - gdf_hhtype['white_family_other'] - gdf_hhtype['black_family_other']
gdf_hhtype['other_nonfamily'] = gdf_hhtype['nonfamily'] - gdf_hhtype['white_nonfamily'] - gdf_hhtype['black_nonfamily']

## white & non-hispanic
gdf['nhisp_white_married_couple_family'] = gdf['B11001He3']
gdf['nhisp_white_family_other'] = gdf['B11001He4']
gdf['nhisp_white_nonfamily'] = gdf['B11001He7']

## hispanic
gdf['hisp_married_couple_family'] = gdf['B11001Ie3']
gdf['hisp_family_other'] = gdf['B11001Ie4']
gdf['hisp_nonfamily'] = gdf['B11001Ie7']
gdf_hhtype = gdf_hhtype.merge(gdf[[value for value in gdf.columns if value=="GEOID" or value.startswith("nhisp") or value.startswith("hisp")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_hhtype = gdf_hhtype.drop(columns=['GEOID'])

## non-hispanic
gdf_hhtype['nhisp_married_couple_family'] = gdf_hhtype['married_couple_family'] - gdf_hhtype['hisp_married_couple_family']
gdf_hhtype['nhisp_family_other'] = gdf_hhtype['family_other'] - gdf_hhtype['hisp_family_other']
gdf_hhtype['nhisp_nonfamily'] = gdf_hhtype['nonfamily'] - gdf_hhtype['hisp_nonfamily']

gdf_hhtype

,GEOID_Data,married_couple_family,family_other,nonfamily,white_married_couple_family,white_family_other,white_nonfamily,black_married_couple_family,black_family_other,black_nonfamily,...,other_nonfamily,nhisp_white_married_couple_family,nhisp_white_family_other,nhisp_white_nonfamily,hisp_married_couple_family,hisp_family_other,hisp_nonfamily,nhisp_married_couple_family,nhisp_family_other,nhisp_nonfamily
0,14000US17031221000,306.0,169.0,588.0,175.0,93.0,449.0,11.0,17.0,37.0,...,102.0,124.0,17.0,355.0,166.0,135.0,165.0,140.0,34.0,423.0
1,14000US17031221100,433.0,277.0,1301.0,327.0,110.0,968.0,0.0,57.0,74.0,...,259.0,242.0,21.0,789.0,145.0,181.0,296.0,288.0,96.0,1005.0
2,14000US17031242800,304.0,75.0,281.0,281.0,70.0,265.0,0.0,0.0,0.0,...,16.0,224.0,65.0,259.0,75.0,10.0,12.0,229.0,65.0,269.0
3,14000US17031242900,348.0,72.0,382.0,257.0,38.0,286.0,23.0,18.0,18.0,...,78.0,251.0,38.0,250.0,45.0,16.0,70.0,303.0,56.0,312.0
4,14000US17031243000,421.0,72.0,586.0,253.0,27.0,492.0,0.0,10.0,0.0,...,94.0,245.0,19.0,492.0,127.0,43.0,9.0,294.0,29.0,577.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,228.0,840.0,1427.0,0.0,0.0,0.0,228.0,794.0,1353.0,...,74.0,0.0,0.0,0.0,0.0,0.0,0.0,228.0,840.0,1427.0
793,14000US17031540101,136.0,952.0,573.0,23.0,23.0,0.0,113.0,902.0,573.0,...,0.0,0.0,0.0,0.0,23.0,35.0,0.0,113.0,917.0,573.0
794,14000US17031050300,643.0,108.0,340.0,465.0,85.0,285.0,0.0,0.0,10.0,...,45.0,447.0,58.0,207.0,69.0,50.0,78.0,574.0,58.0,262.0
795,14000US17031830700,436.0,132.0,1351.0,258.0,33.0,819.0,128.0,66.0,258.0,...,274.0,207.0,0.0,771.0,69.0,33.0,98.0,367.0,99.0,1253.0


In [26]:
gdf_hhtype.loc[:, 'married_couple_family':'nonfamily'] = gdf_hhtype.loc[:, 'married_couple_family':'nonfamily'].div(gdf_hhtype.loc[:, 'married_couple_family':'nonfamily'].sum(axis=1), axis=0)
gdf_hhtype.loc[:, 'white_married_couple_family':'white_nonfamily'] = gdf_hhtype.loc[:, 'white_married_couple_family':'white_nonfamily'].div(gdf_hhtype.loc[:, 'white_married_couple_family':'white_nonfamily'].sum(axis=1), axis=0)
gdf_hhtype.loc[:, 'black_married_couple_family':'black_nonfamily'] = gdf_hhtype.loc[:, 'black_married_couple_family':'black_nonfamily'].div(gdf_hhtype.loc[:, 'black_married_couple_family':'black_nonfamily'].sum(axis=1), axis=0)
gdf_hhtype.loc[:, 'other_married_couple_family':'other_nonfamily'] = gdf_hhtype.loc[:, 'other_married_couple_family':'other_nonfamily'].div(gdf_hhtype.loc[:, 'other_married_couple_family':'other_nonfamily'].sum(axis=1), axis=0)
gdf_hhtype.loc[:, 'nhisp_white_married_couple_family':'nhisp_white_nonfamily'] = gdf_hhtype.loc[:, 'nhisp_white_married_couple_family':'nhisp_white_nonfamily'].div(gdf_hhtype.loc[:, 'nhisp_white_married_couple_family':'nhisp_white_nonfamily'].sum(axis=1), axis=0)
gdf_hhtype.loc[:, 'hisp_married_couple_family':'hisp_nonfamily'] = gdf_hhtype.loc[:, 'hisp_married_couple_family':'hisp_nonfamily'].div(gdf_hhtype.loc[:, 'hisp_married_couple_family':'hisp_nonfamily'].sum(axis=1), axis=0)
gdf_hhtype.loc[:, 'nhisp_married_couple_family':'nhisp_nonfamily'] = gdf_hhtype.loc[:, 'nhisp_married_couple_family':'nhisp_nonfamily'].div(gdf_hhtype.loc[:, 'nhisp_married_couple_family':'nhisp_nonfamily'].sum(axis=1), axis=0)
gdf_hhtype.to_csv("data/chicago/var/household_type.csv", index=False)
gdf_hhtype

,GEOID_Data,married_couple_family,family_other,nonfamily,white_married_couple_family,white_family_other,white_nonfamily,black_married_couple_family,black_family_other,black_nonfamily,...,other_nonfamily,nhisp_white_married_couple_family,nhisp_white_family_other,nhisp_white_nonfamily,hisp_married_couple_family,hisp_family_other,hisp_nonfamily,nhisp_married_couple_family,nhisp_family_other,nhisp_nonfamily
0,14000US17031221000,0.287865,0.158984,0.553151,0.244073,0.129707,0.626220,0.169231,0.261538,0.569231,...,0.362989,0.250000,0.034274,0.715726,0.356223,0.289700,0.354077,0.234506,0.056951,0.708543
1,14000US17031221100,0.215316,0.137742,0.646942,0.232740,0.078292,0.688968,0.000000,0.435115,0.564885,...,0.545263,0.230038,0.019962,0.750000,0.233119,0.290997,0.475884,0.207343,0.069114,0.723542
2,14000US17031242800,0.460606,0.113636,0.425758,0.456169,0.113636,0.430195,NaN,NaN,NaN,...,0.363636,0.408759,0.118613,0.472628,0.773196,0.103093,0.123711,0.406750,0.115453,0.477798
3,14000US17031242900,0.433915,0.089776,0.476309,0.442341,0.065404,0.492255,0.389831,0.305085,0.305085,...,0.481481,0.465677,0.070501,0.463822,0.343511,0.122137,0.534351,0.451565,0.083458,0.464978
4,14000US17031243000,0.390176,0.066728,0.543095,0.327720,0.034974,0.637306,0.000000,1.000000,0.000000,...,0.316498,0.324074,0.025132,0.650794,0.709497,0.240223,0.050279,0.326667,0.032222,0.641111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.091383,0.336673,0.571944,NaN,NaN,NaN,0.096000,0.334316,0.569684,...,0.616667,NaN,NaN,NaN,NaN,NaN,NaN,0.091383,0.336673,0.571944
793,14000US17031540101,0.081878,0.573149,0.344973,0.500000,0.500000,0.000000,0.071159,0.568010,0.360831,...,0.000000,NaN,NaN,NaN,0.396552,0.603448,0.000000,0.070493,0.572052,0.357455
794,14000US17031050300,0.589368,0.098992,0.311641,0.556886,0.101796,0.341317,0.000000,0.000000,1.000000,...,0.182927,0.627809,0.081461,0.290730,0.350254,0.253807,0.395939,0.642058,0.064877,0.293065
795,14000US17031830700,0.227202,0.068786,0.704013,0.232432,0.029730,0.737838,0.283186,0.146018,0.570796,...,0.767507,0.211656,0.000000,0.788344,0.345000,0.165000,0.490000,0.213496,0.057592,0.728912


Mobility

In [53]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B07003") or value.startswith("B07004")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X07_MIGRATION', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['same_house'] = gdf['B07003e4']
gdf['moved_same_county'] = gdf['B07003e7']
gdf['moved_same_state'] = gdf['B07003e10']
gdf['moved_same_country'] = gdf['B07003e13']
gdf['moved_from_abroad'] = gdf['B07003e16']

## white
gdf['white_same_house'] = gdf['B07004Ae2']
gdf['white_moved_same_county'] = gdf['B07004Ae3']
gdf['white_moved_same_state'] = gdf['B07004Ae4']
gdf['white_moved_same_country'] = gdf['B07004Ae5']
gdf['white_moved_from_abroad'] = gdf['B07004Ae6']

## black
gdf['black_same_house'] = gdf['B07004Be2']
gdf['black_moved_same_county'] = gdf['B07004Be3']
gdf['black_moved_same_state'] = gdf['B07004Be4']
gdf['black_moved_same_country'] = gdf['B07004Be5']
gdf['black_moved_from_abroad'] = gdf['B07004Be6']
gdf_mob = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not (value.startswith("B07003") or value.startswith("B07004"))]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_mob = gdf_mob.drop(columns=['GEOID', 'geometry'])

## other races
gdf_mob['other_same_house'] = gdf_mob['same_house'] - gdf_mob['white_same_house'] - gdf_mob['black_same_house']
gdf_mob['other_moved_same_county'] = gdf_mob['moved_same_county'] - gdf_mob['white_moved_same_county'] - gdf_mob['black_moved_same_county']
gdf_mob['other_moved_same_state'] = gdf_mob['moved_same_state'] - gdf_mob['white_moved_same_state'] - gdf_mob['black_moved_same_state']
gdf_mob['other_moved_same_country'] = gdf_mob['moved_same_country'] - gdf_mob['white_moved_same_country'] - gdf_mob['black_moved_same_country']
gdf_mob['other_moved_from_abroad'] = gdf_mob['moved_from_abroad'] - gdf_mob['white_moved_from_abroad'] - gdf_mob['black_moved_from_abroad']

## white & non-hispanic
gdf['nhisp_white_same_house'] = gdf['B07004He2']
gdf['nhisp_white_moved_same_county'] = gdf['B07004He3']
gdf['nhisp_white_moved_same_state'] = gdf['B07004He4']
gdf['nhisp_white_moved_same_country'] = gdf['B07004He5']
gdf['nhisp_white_moved_from_abroad'] = gdf['B07004He6']

## hispanic
gdf['hisp_same_house'] = gdf['B07004Ie2']
gdf['hisp_moved_same_county'] = gdf['B07004Ie3']
gdf['hisp_moved_same_state'] = gdf['B07004Ie4']
gdf['hisp_moved_same_country'] = gdf['B07004Ie5']
gdf['hisp_moved_from_abroad'] = gdf['B07004Ie6']
gdf_mob = gdf_mob.merge(gdf[[value for value in gdf.columns if value=="GEOID" or value.startswith("nhisp") or value.startswith("hisp")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_mob = gdf_mob.drop(columns=['GEOID'])

## non-hispanic
gdf_mob['nhisp_same_house'] = gdf_mob['same_house'] - gdf_mob['hisp_same_house']
gdf_mob['nhisp_moved_same_county'] = gdf_mob['moved_same_county'] - gdf_mob['hisp_moved_same_county']
gdf_mob['nhisp_moved_same_state'] = gdf_mob['moved_same_state'] - gdf_mob['hisp_moved_same_state']
gdf_mob['nhisp_moved_same_country'] = gdf_mob['moved_same_country'] - gdf_mob['hisp_moved_same_country']
gdf_mob['nhisp_moved_from_abroad'] = gdf_mob['moved_from_abroad'] - gdf_mob['hisp_moved_from_abroad']

gdf_mob

,GEOID_Data,same_house,moved_same_county,moved_same_state,moved_same_country,moved_from_abroad,white_same_house,white_moved_same_county,white_moved_same_state,white_moved_same_country,...,hisp_same_house,hisp_moved_same_county,hisp_moved_same_state,hisp_moved_same_country,hisp_moved_from_abroad,nhisp_same_house,nhisp_moved_same_county,nhisp_moved_same_state,nhisp_moved_same_country,nhisp_moved_from_abroad
0,14000US17031221000,2164.0,328.0,23.0,40.0,6.0,1393.0,244.0,23.0,24.0,...,1220.0,58.0,0.0,5.0,0.0,944.0,270.0,23.0,35.0,6.0
1,14000US17031221100,3712.0,658.0,29.0,43.0,39.0,2413.0,388.0,17.0,43.0,...,1512.0,110.0,0.0,0.0,0.0,2200.0,548.0,29.0,43.0,39.0
2,14000US17031242800,1358.0,175.0,0.0,12.0,0.0,1195.0,169.0,0.0,12.0,...,333.0,6.0,0.0,0.0,0.0,1025.0,169.0,0.0,12.0,0.0
3,14000US17031242900,1582.0,167.0,15.0,67.0,10.0,1045.0,90.0,15.0,65.0,...,401.0,10.0,12.0,0.0,10.0,1181.0,157.0,3.0,67.0,0.0
4,14000US17031243000,1739.0,345.0,63.0,28.0,9.0,1118.0,202.0,14.0,28.0,...,490.0,33.0,12.0,0.0,0.0,1249.0,312.0,51.0,28.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,4245.0,1705.0,0.0,70.0,0.0,24.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4245.0,1705.0,0.0,70.0,0.0
793,14000US17031540101,3758.0,596.0,86.0,12.0,0.0,69.0,84.0,10.0,0.0,...,104.0,76.0,10.0,0.0,0.0,3654.0,520.0,76.0,12.0,0.0
794,14000US17031050300,2539.0,141.0,19.0,43.0,0.0,1598.0,107.0,19.0,43.0,...,453.0,27.0,9.0,0.0,0.0,2086.0,114.0,10.0,43.0,0.0
795,14000US17031830700,3284.0,444.0,71.0,141.0,45.0,1526.0,275.0,50.0,90.0,...,346.0,143.0,0.0,0.0,9.0,2938.0,301.0,71.0,141.0,36.0


In [54]:
gdf_mob.loc[:, 'same_house':'moved_from_abroad'] = gdf_mob.loc[:, 'same_house':'moved_from_abroad'].div(gdf_mob.loc[:, 'same_house':'moved_from_abroad'].sum(axis=1), axis=0)
gdf_mob.loc[:, 'white_same_house':'white_moved_from_abroad'] = gdf_mob.loc[:, 'white_same_house':'white_moved_from_abroad'].div(gdf_mob.loc[:, 'white_same_house':'white_moved_from_abroad'].sum(axis=1), axis=0)
gdf_mob.loc[:, 'black_same_house':'black_moved_from_abroad'] = gdf_mob.loc[:, 'black_same_house':'black_moved_from_abroad'].div(gdf_mob.loc[:, 'black_same_house':'black_moved_from_abroad'].sum(axis=1), axis=0)
gdf_mob.loc[:, 'other_same_house':'other_moved_from_abroad'] = gdf_mob.loc[:, 'other_same_house':'other_moved_from_abroad'].div(gdf_mob.loc[:, 'other_same_house':'other_moved_from_abroad'].sum(axis=1), axis=0)
gdf_mob.loc[:, 'nhisp_white_same_house':'nhisp_white_moved_from_abroad'] = gdf_mob.loc[:, 'nhisp_white_same_house':'nhisp_white_moved_from_abroad'].div(gdf_mob.loc[:, 'nhisp_white_same_house':'nhisp_white_moved_from_abroad'].sum(axis=1), axis=0)
gdf_mob.loc[:, 'hisp_same_house':'hisp_moved_from_abroad'] = gdf_mob.loc[:, 'hisp_same_house':'hisp_moved_from_abroad'].div(gdf_mob.loc[:, 'hisp_same_house':'hisp_moved_from_abroad'].sum(axis=1), axis=0)
gdf_mob.loc[:, 'nhisp_same_house':'nhisp_moved_from_abroad'] = gdf_mob.loc[:, 'nhisp_same_house':'nhisp_moved_from_abroad'].div(gdf_mob.loc[:, 'nhisp_same_house':'nhisp_moved_from_abroad'].sum(axis=1), axis=0)
gdf_mob.to_csv("data/chicago/var/mobility.csv", index=False)
gdf_mob

,GEOID_Data,same_house,moved_same_county,moved_same_state,moved_same_country,moved_from_abroad,white_same_house,white_moved_same_county,white_moved_same_state,white_moved_same_country,...,hisp_same_house,hisp_moved_same_county,hisp_moved_same_state,hisp_moved_same_country,hisp_moved_from_abroad,nhisp_same_house,nhisp_moved_same_county,nhisp_moved_same_state,nhisp_moved_same_country,nhisp_moved_from_abroad
0,14000US17031221000,0.844982,0.128075,0.008981,0.015619,0.002343,0.824260,0.144379,0.013609,0.014201,...,0.950896,0.045207,0.000000,0.003897,0.000000,0.738654,0.211268,0.017997,0.027387,0.004695
1,14000US17031221100,0.828387,0.146842,0.006472,0.009596,0.008703,0.832069,0.133793,0.005862,0.014828,...,0.932182,0.067818,0.000000,0.000000,0.000000,0.769500,0.191675,0.010143,0.015040,0.013641
2,14000US17031242800,0.878964,0.113269,0.000000,0.007767,0.000000,0.868459,0.122820,0.000000,0.008721,...,0.982301,0.017699,0.000000,0.000000,0.000000,0.849917,0.140133,0.000000,0.009950,0.000000
3,14000US17031242900,0.859316,0.090712,0.008148,0.036393,0.005432,0.860082,0.074074,0.012346,0.053498,...,0.926097,0.023095,0.027714,0.000000,0.023095,0.838778,0.111506,0.002131,0.047585,0.000000
4,14000US17031243000,0.796245,0.157967,0.028846,0.012821,0.004121,0.815463,0.147338,0.010212,0.020423,...,0.915888,0.061682,0.022430,0.000000,0.000000,0.757429,0.189206,0.030928,0.016980,0.005458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.705150,0.283223,0.000000,0.011628,0.000000,1.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.705150,0.283223,0.000000,0.011628,0.000000
793,14000US17031540101,0.844115,0.133872,0.019317,0.002695,0.000000,0.423313,0.515337,0.061350,0.000000,...,0.547368,0.400000,0.052632,0.000000,0.000000,0.857344,0.122008,0.017832,0.002816,0.000000
794,14000US17031050300,0.925966,0.051422,0.006929,0.015682,0.000000,0.904358,0.060555,0.010753,0.024335,...,0.926380,0.055215,0.018405,0.000000,0.000000,0.925877,0.050599,0.004439,0.019086,0.000000
795,14000US17031830700,0.824090,0.111418,0.017817,0.035383,0.011292,0.786193,0.141680,0.025760,0.046368,...,0.694779,0.287149,0.000000,0.000000,0.018072,0.842558,0.086321,0.020361,0.040436,0.010324


Tenure

In [11]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B25003")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X25_HOUSING_CHARACTERISTICS', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['owner'] = gdf['B25003e2']
gdf['renter'] = gdf['B25003e3']

## white
gdf['white_owner'] = gdf['B25003Ae2']
gdf['white_renter'] = gdf['B25003Ae3']

## black
gdf['black_owner'] = gdf['B25003Be2']
gdf['black_renter'] = gdf['B25003Be3']
gdf_tnr = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not value.startswith("B25003")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_tnr = gdf_tnr.drop(columns=['GEOID', 'geometry'])

## other races
gdf_tnr['other_owner'] = gdf_tnr['owner'] - gdf_tnr['white_owner'] - gdf_tnr['black_owner']
gdf_tnr['other_renter'] = gdf_tnr['renter'] - gdf_tnr['white_renter'] - gdf_tnr['black_renter']

## white & non-hispanic
gdf['nhisp_white_owner'] = gdf['B25003He2']
gdf['nhisp_white_renter'] = gdf['B25003He3']

## hispanic
gdf['hisp_owner'] = gdf['B25003Ie2']
gdf['hisp_renter'] = gdf['B25003Ie3']
gdf_tnr = gdf_tnr.merge(gdf[[value for value in gdf.columns if value=="GEOID" or value.startswith("nhisp") or value.startswith("hisp")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_tnr = gdf_tnr.drop(columns=['GEOID'])

## non-hispanic
gdf_tnr['nhisp_owner'] = gdf_tnr['owner'] - gdf_tnr['hisp_owner']
gdf_tnr['nhisp_renter'] = gdf_tnr['renter'] - gdf_tnr['hisp_renter']

gdf_tnr

,GEOID_Data,owner,renter,white_owner,white_renter,black_owner,black_renter,other_owner,other_renter,nhisp_white_owner,nhisp_white_renter,hisp_owner,hisp_renter,nhisp_owner,nhisp_renter
0,14000US17031221000,343.0,720.0,259.0,458.0,20.0,45.0,64.0,217.0,163.0,333.0,150.0,316.0,193.0,404.0
1,14000US17031221100,625.0,1386.0,443.0,962.0,0.0,131.0,182.0,293.0,357.0,695.0,151.0,471.0,474.0,915.0
2,14000US17031242800,384.0,276.0,356.0,260.0,0.0,0.0,28.0,16.0,309.0,239.0,60.0,37.0,324.0,239.0
3,14000US17031242900,430.0,372.0,317.0,264.0,23.0,36.0,90.0,72.0,311.0,228.0,54.0,77.0,376.0,295.0
4,14000US17031243000,566.0,513.0,351.0,421.0,10.0,0.0,205.0,92.0,335.0,421.0,142.0,37.0,424.0,476.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,512.0,1983.0,0.0,0.0,414.0,1961.0,98.0,22.0,0.0,0.0,0.0,0.0,512.0,1983.0
793,14000US17031540101,88.0,1573.0,8.0,38.0,68.0,1520.0,12.0,15.0,0.0,0.0,20.0,38.0,68.0,1535.0
794,14000US17031050300,675.0,416.0,469.0,366.0,0.0,10.0,206.0,40.0,432.0,280.0,111.0,86.0,564.0,330.0
795,14000US17031830700,498.0,1421.0,362.0,748.0,51.0,401.0,85.0,272.0,362.0,616.0,11.0,189.0,487.0,1232.0


In [13]:
gdf_tnr.loc[:, 'owner':'renter'] = gdf_tnr.loc[:, 'owner':'renter'].div(gdf_tnr.loc[:, 'owner':'renter'].sum(axis=1), axis=0)
gdf_tnr.loc[:, 'white_owner':'white_renter'] = gdf_tnr.loc[:, 'white_owner':'white_renter'].div(gdf_tnr.loc[:, 'white_owner':'white_renter'].sum(axis=1), axis=0)
gdf_tnr.loc[:, 'black_owner':'black_renter'] = gdf_tnr.loc[:, 'black_owner':'black_renter'].div(gdf_tnr.loc[:, 'black_owner':'black_renter'].sum(axis=1), axis=0)
gdf_tnr.loc[:, 'other_owner':'other_renter'] = gdf_tnr.loc[:, 'other_owner':'other_renter'].div(gdf_tnr.loc[:, 'other_owner':'other_renter'].sum(axis=1), axis=0)
gdf_tnr.loc[:, 'nhisp_white_owner':'nhisp_white_renter'] = gdf_tnr.loc[:, 'nhisp_white_owner':'nhisp_white_renter'].div(gdf_tnr.loc[:, 'nhisp_white_owner':'nhisp_white_renter'].sum(axis=1), axis=0)
gdf_tnr.loc[:, 'hisp_owner':'hisp_renter'] = gdf_tnr.loc[:, 'hisp_owner':'hisp_renter'].div(gdf_tnr.loc[:, 'hisp_owner':'hisp_renter'].sum(axis=1), axis=0)
gdf_tnr.loc[:, 'nhisp_owner':'nhisp_renter'] = gdf_tnr.loc[:, 'nhisp_owner':'nhisp_renter'].div(gdf_tnr.loc[:, 'nhisp_owner':'nhisp_renter'].sum(axis=1), axis=0)
gdf_tnr.to_csv("data/chicago/var/tenure.csv", index=False)
gdf_tnr

,GEOID_Data,owner,renter,white_owner,white_renter,black_owner,black_renter,other_owner,other_renter,nhisp_white_owner,nhisp_white_renter,hisp_owner,hisp_renter,nhisp_owner,nhisp_renter
0,14000US17031221000,0.322672,0.677328,0.361227,0.638773,0.307692,0.692308,0.227758,0.772242,0.328629,0.671371,0.321888,0.678112,0.323283,0.676717
1,14000US17031221100,0.310791,0.689209,0.315302,0.684698,0.000000,1.000000,0.383158,0.616842,0.339354,0.660646,0.242765,0.757235,0.341253,0.658747
2,14000US17031242800,0.581818,0.418182,0.577922,0.422078,NaN,NaN,0.636364,0.363636,0.563869,0.436131,0.618557,0.381443,0.575488,0.424512
3,14000US17031242900,0.536160,0.463840,0.545611,0.454389,0.389831,0.610169,0.555556,0.444444,0.576994,0.423006,0.412214,0.587786,0.560358,0.439642
4,14000US17031243000,0.524560,0.475440,0.454663,0.545337,1.000000,0.000000,0.690236,0.309764,0.443122,0.556878,0.793296,0.206704,0.471111,0.528889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.205210,0.794790,NaN,NaN,0.174316,0.825684,0.816667,0.183333,NaN,NaN,NaN,NaN,0.205210,0.794790
793,14000US17031540101,0.052980,0.947020,0.173913,0.826087,0.042821,0.957179,0.444444,0.555556,NaN,NaN,0.344828,0.655172,0.042420,0.957580
794,14000US17031050300,0.618698,0.381302,0.561677,0.438323,0.000000,1.000000,0.837398,0.162602,0.606742,0.393258,0.563452,0.436548,0.630872,0.369128
795,14000US17031830700,0.259510,0.740490,0.326126,0.673874,0.112832,0.887168,0.238095,0.761905,0.370143,0.629857,0.055000,0.945000,0.283304,0.716696


Occupants per room

In [38]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B25014")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X25_HOUSING_CHARACTERISTICS', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['one_or_less'] = gdf[['B25014Ae2', 'B25014Be2', 'B25014Ce2', 'B25014De2', 'B25014Ee2', 'B25014Fe2', 'B25014Ge2']].sum(axis=1)
gdf['more_than_one'] = gdf[['B25014Ae3', 'B25014Be3', 'B25014Ce3', 'B25014De3', 'B25014Ee3', 'B25014Fe3', 'B25014Ge3']].sum(axis=1)

## white
gdf['white_one_or_less'] = gdf['B25014Ae2']
gdf['white_more_than_one'] = gdf['B25014Ae3']

## black
gdf['black_one_or_less'] = gdf['B25014Be2']
gdf['black_more_than_one'] = gdf['B25014Be3']

## other races
gdf['other_one_or_less'] = gdf[['B25014Ce2', 'B25014De2', 'B25014Ee2', 'B25014Fe2', 'B25014Ge2']].sum(axis=1)
gdf['other_more_than_one'] = gdf[['B25014Ce3', 'B25014De3', 'B25014Ee3', 'B25014Fe3', 'B25014Ge3']].sum(axis=1)

## white & non-hispanic
gdf['nhisp_white_one_or_less'] = gdf['B25014He2']
gdf['nhisp_white_more_than_one'] = gdf['B25014He3']

## hispanic
gdf['hisp_one_or_less'] = gdf['B25014Ie2']
gdf['hisp_more_than_one'] = gdf['B25014Ie3']
gdf_ocr = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not value.startswith("B25014")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_ocr = gdf_ocr.drop(columns=['GEOID', 'geometry'])

## non-hispanic
gdf_ocr['nhisp_one_or_less'] = gdf_ocr['one_or_less'] - gdf_ocr['hisp_one_or_less']
gdf_ocr['nhisp_more_than_one'] = gdf_ocr['more_than_one'] - gdf_ocr['hisp_more_than_one']

gdf_ocr

,GEOID_Data,one_or_less,more_than_one,white_one_or_less,white_more_than_one,black_one_or_less,black_more_than_one,other_one_or_less,other_more_than_one,nhisp_white_one_or_less,nhisp_white_more_than_one,hisp_one_or_less,hisp_more_than_one,nhisp_one_or_less,nhisp_more_than_one
0,14000US17031221000,1059.0,4.0,713.0,4.0,65.0,0.0,281.0,0.0,496.0,0.0,462.0,4.0,597.0,0.0
1,14000US17031221100,1985.0,26.0,1379.0,26.0,131.0,0.0,475.0,0.0,1052.0,0.0,596.0,26.0,1389.0,0.0
2,14000US17031242800,646.0,14.0,602.0,14.0,0.0,0.0,44.0,0.0,534.0,14.0,97.0,0.0,549.0,14.0
3,14000US17031242900,765.0,37.0,574.0,7.0,59.0,0.0,132.0,30.0,532.0,7.0,118.0,13.0,647.0,24.0
4,14000US17031243000,1071.0,8.0,772.0,0.0,10.0,0.0,289.0,8.0,756.0,0.0,179.0,0.0,892.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,2446.0,49.0,0.0,0.0,2326.0,49.0,120.0,0.0,0.0,0.0,0.0,0.0,2446.0,49.0
793,14000US17031540101,1513.0,148.0,46.0,0.0,1440.0,148.0,27.0,0.0,0.0,0.0,58.0,0.0,1455.0,148.0
794,14000US17031050300,1091.0,0.0,835.0,0.0,10.0,0.0,246.0,0.0,712.0,0.0,197.0,0.0,894.0,0.0
795,14000US17031830700,1813.0,106.0,1064.0,46.0,410.0,42.0,339.0,18.0,958.0,20.0,174.0,26.0,1639.0,80.0


In [39]:
gdf_ocr.loc[:, 'one_or_less':'more_than_one'] = gdf_ocr.loc[:, 'one_or_less':'more_than_one'].div(gdf_ocr.loc[:, 'one_or_less':'more_than_one'].sum(axis=1), axis=0)
gdf_ocr.loc[:, 'white_one_or_less':'white_more_than_one'] = gdf_ocr.loc[:, 'white_one_or_less':'white_more_than_one'].div(gdf_ocr.loc[:, 'white_one_or_less':'white_more_than_one'].sum(axis=1), axis=0)
gdf_ocr.loc[:, 'black_one_or_less':'black_more_than_one'] = gdf_ocr.loc[:, 'black_one_or_less':'black_more_than_one'].div(gdf_ocr.loc[:, 'black_one_or_less':'black_more_than_one'].sum(axis=1), axis=0)
gdf_ocr.loc[:, 'other_one_or_less':'other_more_than_one'] = gdf_ocr.loc[:, 'other_one_or_less':'other_more_than_one'].div(gdf_ocr.loc[:, 'other_one_or_less':'other_more_than_one'].sum(axis=1), axis=0)
gdf_ocr.loc[:, 'nhisp_white_one_or_less':'nhisp_white_more_than_one'] = gdf_ocr.loc[:, 'nhisp_white_one_or_less':'nhisp_white_more_than_one'].div(gdf_ocr.loc[:, 'nhisp_white_one_or_less':'nhisp_white_more_than_one'].sum(axis=1), axis=0)
gdf_ocr.loc[:, 'hisp_one_or_less':'hisp_more_than_one'] = gdf_ocr.loc[:, 'hisp_one_or_less':'hisp_more_than_one'].div(gdf_ocr.loc[:, 'hisp_one_or_less':'hisp_more_than_one'].sum(axis=1), axis=0)
gdf_ocr.loc[:, 'nhisp_one_or_less':'nhisp_more_than_one'] = gdf_ocr.loc[:, 'nhisp_one_or_less':'nhisp_more_than_one'].div(gdf_ocr.loc[:, 'nhisp_one_or_less':'nhisp_more_than_one'].sum(axis=1), axis=0)
gdf_ocr.to_csv("data/chicago/var/occupants_per_room.csv", index=False)
gdf_ocr

,GEOID_Data,one_or_less,more_than_one,white_one_or_less,white_more_than_one,black_one_or_less,black_more_than_one,other_one_or_less,other_more_than_one,nhisp_white_one_or_less,nhisp_white_more_than_one,hisp_one_or_less,hisp_more_than_one,nhisp_one_or_less,nhisp_more_than_one
0,14000US17031221000,0.996237,0.003763,0.994421,0.005579,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.991416,0.008584,1.000000,0.000000
1,14000US17031221100,0.987071,0.012929,0.981495,0.018505,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.958199,0.041801,1.000000,0.000000
2,14000US17031242800,0.978788,0.021212,0.977273,0.022727,NaN,NaN,1.000000,0.000000,0.974453,0.025547,1.000000,0.000000,0.975133,0.024867
3,14000US17031242900,0.953865,0.046135,0.987952,0.012048,1.000000,0.000000,0.814815,0.185185,0.987013,0.012987,0.900763,0.099237,0.964232,0.035768
4,14000US17031243000,0.992586,0.007414,1.000000,0.000000,1.000000,0.000000,0.973064,0.026936,1.000000,0.000000,1.000000,0.000000,0.991111,0.008889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.980361,0.019639,NaN,NaN,0.979368,0.020632,1.000000,0.000000,NaN,NaN,NaN,NaN,0.980361,0.019639
793,14000US17031540101,0.910897,0.089103,1.000000,0.000000,0.906801,0.093199,1.000000,0.000000,NaN,NaN,1.000000,0.000000,0.907673,0.092327
794,14000US17031050300,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
795,14000US17031830700,0.944763,0.055237,0.958559,0.041441,0.907080,0.092920,0.949580,0.050420,0.979550,0.020450,0.870000,0.130000,0.953461,0.046539


Units in structure

In [30]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B25032")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X25_HOUSING_CHARACTERISTICS', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['single_family'] = gdf[[f'B25032e{i}' for i in range(3, 5)]].sum(axis=1) + gdf[[f'B25032e{i}' for i in range(14, 16)]].sum(axis=1)
gdf['multi_family'] = gdf[[f'B25032e{i}' for i in range(5, 11)]].sum(axis=1) + gdf[[f'B25032e{i}' for i in range(16, 22)]].sum(axis=1)
gdf['units_other'] = gdf[[f'B25032e{i}' for i in range(11, 13)]].sum(axis=1) + gdf[[f'B25032e{i}' for i in range(22, 24)]].sum(axis=1)

## white
gdf['white_single_family'] = gdf[[f'B25032Ae{i}' for i in range(2, 4)]].sum(axis=1)
gdf['white_multi_family'] = gdf[[f'B25032Ae{i}' for i in range(4, 10)]].sum(axis=1)
gdf['white_units_other'] = gdf[[f'B25032Ae{i}' for i in range(10, 12)]].sum(axis=1)

## black
gdf['black_single_family'] = gdf[[f'B25032Be{i}' for i in range(2, 4)]].sum(axis=1)
gdf['black_multi_family'] = gdf[[f'B25032Be{i}' for i in range(4, 10)]].sum(axis=1)
gdf['black_units_other'] = gdf[[f'B25032Be{i}' for i in range(10, 12)]].sum(axis=1)
gdf_unit = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not value.startswith("B25032")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_unit = gdf_unit.drop(columns=['GEOID', 'geometry'])

## other races
gdf_unit['other_single_family'] = gdf_unit['single_family'] - gdf_unit['white_single_family'] - gdf_unit['black_single_family']
gdf_unit['other_multi_family'] = gdf_unit['multi_family'] - gdf_unit['white_multi_family'] - gdf_unit['black_multi_family']
gdf_unit['units_other_other'] = gdf_unit['units_other'] - gdf_unit['white_units_other'] - gdf_unit['black_units_other']

## white & non-hispanic
gdf['nhisp_white_single_family'] = gdf[[f'B25032He{i}' for i in range(2, 4)]].sum(axis=1)
gdf['nhisp_white_multi_family'] = gdf[[f'B25032He{i}' for i in range(4, 10)]].sum(axis=1)
gdf['nhisp_white_units_other'] = gdf[[f'B25032He{i}' for i in range(10, 12)]].sum(axis=1)

## hispanic
gdf['hisp_single_family'] = gdf[[f'B25032Ie{i}' for i in range(2, 4)]].sum(axis=1)
gdf['hisp_multi_family'] = gdf[[f'B25032Ie{i}' for i in range(4, 10)]].sum(axis=1)
gdf['hisp_units_other'] = gdf[[f'B25032Ie{i}' for i in range(10, 12)]].sum(axis=1)
gdf_unit = gdf_unit.merge(gdf[[value for value in gdf.columns if value=="GEOID" or value.startswith("nhisp") or value.startswith("hisp")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_unit = gdf_unit.drop(columns=['GEOID'])

## non-hispanic
gdf_unit['nhisp_single_family'] = gdf_unit['single_family'] - gdf_unit['hisp_single_family']
gdf_unit['nhisp_multi_family'] = gdf_unit['multi_family'] - gdf_unit['hisp_multi_family']
gdf_unit['nhisp_units_other'] = gdf_unit['units_other'] - gdf_unit['hisp_units_other']

gdf_unit

,GEOID_Data,single_family,multi_family,units_other,white_single_family,white_multi_family,white_units_other,black_single_family,black_multi_family,black_units_other,...,other_units_other,nhisp_white_single_family,nhisp_white_multi_family,nhisp_white_units_other,hisp_single_family,hisp_multi_family,hisp_units_other,nhisp_single_family,nhisp_multi_family,nhisp_units_other
0,14000US17031221000,156.0,907.0,0.0,121.0,596.0,0.0,0.0,65.0,0.0,...,0.0,80.0,416.0,0.0,71.0,395.0,0.0,85.0,512.0,0.0
1,14000US17031221100,230.0,1781.0,0.0,127.0,1278.0,0.0,0.0,131.0,0.0,...,0.0,103.0,949.0,0.0,76.0,546.0,0.0,154.0,1235.0,0.0
2,14000US17031242800,219.0,441.0,0.0,214.0,402.0,0.0,0.0,0.0,0.0,...,0.0,188.0,360.0,0.0,26.0,71.0,0.0,193.0,370.0,0.0
3,14000US17031242900,168.0,634.0,0.0,129.0,452.0,0.0,23.0,36.0,0.0,...,0.0,129.0,410.0,0.0,28.0,103.0,0.0,140.0,531.0,0.0
4,14000US17031243000,294.0,777.0,8.0,247.0,525.0,0.0,0.0,10.0,0.0,...,8.0,239.0,517.0,0.0,23.0,156.0,0.0,271.0,621.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,310.0,2185.0,0.0,0.0,0.0,0.0,236.0,2139.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,310.0,2185.0,0.0
793,14000US17031540101,1203.0,458.0,0.0,31.0,15.0,0.0,1157.0,431.0,0.0,...,0.0,0.0,0.0,0.0,31.0,27.0,0.0,1172.0,431.0,0.0
794,14000US17031050300,531.0,548.0,12.0,403.0,432.0,0.0,0.0,10.0,0.0,...,12.0,392.0,320.0,0.0,73.0,112.0,12.0,458.0,436.0,0.0
795,14000US17031830700,74.0,1845.0,0.0,49.0,1061.0,0.0,7.0,445.0,0.0,...,0.0,49.0,929.0,0.0,11.0,189.0,0.0,63.0,1656.0,0.0


In [31]:
gdf_unit.loc[:, 'single_family':'units_other'] = gdf_unit.loc[:, 'single_family':'units_other'].div(gdf_unit.loc[:, 'single_family':'units_other'].sum(axis=1), axis=0)
gdf_unit.loc[:, 'white_single_family':'white_units_other'] = gdf_unit.loc[:, 'white_single_family':'white_units_other'].div(gdf_unit.loc[:, 'white_single_family':'white_units_other'].sum(axis=1), axis=0)
gdf_unit.loc[:, 'black_single_family':'black_units_other'] = gdf_unit.loc[:, 'black_single_family':'black_units_other'].div(gdf_unit.loc[:, 'black_single_family':'black_units_other'].sum(axis=1), axis=0)
gdf_unit.loc[:, 'other_single_family':'other_units_other'] = gdf_unit.loc[:, 'other_single_family':'other_units_other'].div(gdf_unit.loc[:, 'other_single_family':'other_units_other'].sum(axis=1), axis=0)
gdf_unit.loc[:, 'nhisp_white_single_family':'nhisp_white_units_other'] = gdf_unit.loc[:, 'nhisp_white_single_family':'nhisp_white_units_other'].div(gdf_unit.loc[:, 'nhisp_white_single_family':'nhisp_white_units_other'].sum(axis=1), axis=0)
gdf_unit.loc[:, 'hisp_single_family':'hisp_units_other'] = gdf_unit.loc[:, 'hisp_single_family':'hisp_units_other'].div(gdf_unit.loc[:, 'hisp_single_family':'hisp_units_other'].sum(axis=1), axis=0)
gdf_unit.loc[:, 'nhisp_single_family':'nhisp_units_other'] = gdf_unit.loc[:, 'nhisp_single_family':'nhisp_units_other'].div(gdf_unit.loc[:, 'nhisp_single_family':'nhisp_units_other'].sum(axis=1), axis=0)
gdf_unit.to_csv("data/chicago/var/units_per_structure.csv", index=False)
gdf_unit

,GEOID_Data,single_family,multi_family,units_other,white_single_family,white_multi_family,white_units_other,black_single_family,black_multi_family,black_units_other,...,other_units_other,nhisp_white_single_family,nhisp_white_multi_family,nhisp_white_units_other,hisp_single_family,hisp_multi_family,hisp_units_other,nhisp_single_family,nhisp_multi_family,nhisp_units_other
0,14000US17031221000,0.146754,0.853246,0.000000,0.168759,0.831241,0.0,0.000000,1.000000,0.0,...,0.000000,0.161290,0.838710,0.0,0.152361,0.847639,0.000000,0.142379,0.857621,0.000000
1,14000US17031221100,0.114371,0.885629,0.000000,0.090391,0.909609,0.0,0.000000,1.000000,0.0,...,0.000000,0.097909,0.902091,0.0,0.122186,0.877814,0.000000,0.110871,0.889129,0.000000
2,14000US17031242800,0.331818,0.668182,0.000000,0.347403,0.652597,0.0,NaN,NaN,NaN,...,0.000000,0.343066,0.656934,0.0,0.268041,0.731959,0.000000,0.342806,0.657194,0.000000
3,14000US17031242900,0.209476,0.790524,0.000000,0.222031,0.777969,0.0,0.389831,0.610169,0.0,...,0.000000,0.239332,0.760668,0.0,0.213740,0.786260,0.000000,0.208644,0.791356,0.000000
4,14000US17031243000,0.272475,0.720111,0.007414,0.319948,0.680052,0.0,0.000000,1.000000,0.0,...,0.026936,0.316138,0.683862,0.0,0.128492,0.871508,0.000000,0.301111,0.690000,0.008889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.124248,0.875752,0.000000,NaN,NaN,NaN,0.099368,0.900632,0.0,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.124248,0.875752,0.000000
793,14000US17031540101,0.724262,0.275738,0.000000,0.673913,0.326087,0.0,0.728589,0.271411,0.0,...,0.000000,NaN,NaN,NaN,0.534483,0.465517,0.000000,0.731129,0.268871,0.000000
794,14000US17031050300,0.486709,0.502291,0.010999,0.482635,0.517365,0.0,0.000000,1.000000,0.0,...,0.048780,0.550562,0.449438,0.0,0.370558,0.568528,0.060914,0.512304,0.487696,0.000000
795,14000US17031830700,0.038562,0.961438,0.000000,0.044144,0.955856,0.0,0.015487,0.984513,0.0,...,0.000000,0.050102,0.949898,0.0,0.055000,0.945000,0.000000,0.036649,0.963351,0.000000


Means of transportation (mode)

In [27]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B08101") or value.startswith("B08105")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X08_COMMUTING', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['car'] = gdf['B08101e9'] + gdf['B08101e17']
gdf['pub_trans'] = gdf['B08101e25']
gdf['means_other'] = gdf['B08101e33'] + gdf['B08101e41'] + gdf['B08101e49']

## white
gdf['white_car'] = gdf['B08105Ae2'] + gdf['B08105Ae3']
gdf['white_pub_trans'] = gdf['B08105Ae4']
gdf['white_means_other'] = gdf['B08105Ae5'] + gdf['B08105Ae6'] + gdf['B08105Ae7']

## black
gdf['black_car'] = gdf['B08105Be2'] + gdf['B08105Be3']
gdf['black_pub_trans'] = gdf['B08105Be4']
gdf['black_means_other'] = gdf['B08105Be5'] + gdf['B08105Be6'] + gdf['B08105Be7']
gdf_mode = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not (value.startswith("B08101") or value.startswith("B08105"))]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_mode = gdf_mode.drop(columns=['GEOID', 'geometry'])

## other races
gdf_mode['other_car'] = gdf_mode['car'] - gdf_mode['white_car'] - gdf_mode['black_car']
gdf_mode['other_pub_trans'] = gdf_mode['pub_trans'] - gdf_mode['white_pub_trans'] - gdf_mode['black_pub_trans']
gdf_mode['other_means_other'] = gdf_mode['means_other'] - gdf_mode['white_means_other'] - gdf_mode['black_means_other']

## white & non-hispanic
gdf['nhisp_white_car'] = gdf['B08105He2'] + gdf['B08105He3']
gdf['nhisp_white_pub_trans'] = gdf['B08105He4']
gdf['nhisp_white_means_other'] = gdf['B08105He5'] + gdf['B08105He6'] + gdf['B08105He7']

## hispanic
gdf['hisp_car'] = gdf['B08105Ie2'] + gdf['B08105Ie3']
gdf['hisp_pub_trans'] = gdf['B08105Ie4']
gdf['hisp_means_other'] = gdf['B08105Ie5'] + gdf['B08105Ie6'] + gdf['B08105Ie7']
gdf_mode = gdf_mode.merge(gdf[[value for value in gdf.columns if value=="GEOID" or value.startswith("nhisp") or value.startswith("hisp")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_mode = gdf_mode.drop(columns=['GEOID'])

## non-hispanic
gdf_mode['nhisp_car'] = gdf_mode['car'] - gdf_mode['hisp_car']
gdf_mode['nhisp_pub_trans'] = gdf_mode['pub_trans'] - gdf_mode['hisp_pub_trans']
gdf_mode['nhisp_means_other'] = gdf_mode['means_other'] - gdf_mode['hisp_means_other']

gdf_mode

,GEOID_Data,car,pub_trans,means_other,white_car,white_pub_trans,white_means_other,black_car,black_pub_trans,black_means_other,...,other_means_other,nhisp_white_car,nhisp_white_pub_trans,nhisp_white_means_other,hisp_car,hisp_pub_trans,hisp_means_other,nhisp_car,nhisp_pub_trans,nhisp_means_other
0,14000US17031221000,946.0,365.0,320.0,576.0,260.0,256.0,46.0,45.0,47.0,...,17.0,287.0,191.0,228.0,571.0,115.0,40.0,375.0,250.0,280.0
1,14000US17031221100,1053.0,887.0,777.0,673.0,587.0,599.0,104.0,54.0,0.0,...,178.0,507.0,467.0,563.0,328.0,165.0,83.0,725.0,722.0,694.0
2,14000US17031242800,485.0,187.0,313.0,452.0,180.0,280.0,6.0,0.0,6.0,...,27.0,375.0,135.0,268.0,94.0,52.0,34.0,391.0,135.0,279.0
3,14000US17031242900,661.0,206.0,399.0,465.0,178.0,245.0,23.0,18.0,26.0,...,128.0,453.0,154.0,225.0,129.0,28.0,80.0,532.0,178.0,319.0
4,14000US17031243000,740.0,334.0,355.0,511.0,273.0,275.0,0.0,8.0,10.0,...,70.0,478.0,261.0,275.0,116.0,30.0,26.0,624.0,304.0,329.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,1097.0,800.0,396.0,24.0,0.0,0.0,1073.0,800.0,322.0,...,74.0,24.0,0.0,0.0,0.0,0.0,0.0,1097.0,800.0,396.0
793,14000US17031540101,716.0,370.0,70.0,43.0,0.0,0.0,673.0,370.0,58.0,...,12.0,0.0,0.0,0.0,43.0,0.0,12.0,673.0,370.0,58.0
794,14000US17031050300,931.0,241.0,555.0,714.0,182.0,294.0,6.0,0.0,10.0,...,251.0,639.0,141.0,283.0,154.0,53.0,57.0,777.0,188.0,498.0
795,14000US17031830700,819.0,955.0,617.0,406.0,678.0,412.0,252.0,134.0,124.0,...,81.0,398.0,570.0,344.0,64.0,123.0,94.0,755.0,832.0,523.0


In [28]:
gdf_mode.loc[:, 'car':'means_other'] = gdf_mode.loc[:, 'car':'means_other'].div(gdf_mode.loc[:, 'car':'means_other'].sum(axis=1), axis=0)
gdf_mode.loc[:, 'white_car':'white_means_other'] = gdf_mode.loc[:, 'white_car':'white_means_other'].div(gdf_mode.loc[:, 'white_car':'white_means_other'].sum(axis=1), axis=0)
gdf_mode.loc[:, 'black_car':'black_means_other'] = gdf_mode.loc[:, 'black_car':'black_means_other'].div(gdf_mode.loc[:, 'black_car':'black_means_other'].sum(axis=1), axis=0)
gdf_mode.loc[:, 'other_car':'other_means_other'] = gdf_mode.loc[:, 'other_car':'other_means_other'].div(gdf_mode.loc[:, 'other_car':'other_means_other'].sum(axis=1), axis=0)
gdf_mode.loc[:, 'nhisp_white_car':'nhisp_white_means_other'] = gdf_mode.loc[:, 'nhisp_white_car':'nhisp_white_means_other'].div(gdf_mode.loc[:, 'nhisp_white_car':'nhisp_white_means_other'].sum(axis=1), axis=0)
gdf_mode.loc[:, 'hisp_car':'hisp_means_other'] = gdf_mode.loc[:, 'hisp_car':'hisp_means_other'].div(gdf_mode.loc[:, 'hisp_car':'hisp_means_other'].sum(axis=1), axis=0)
gdf_mode.loc[:, 'nhisp_car':'nhisp_means_other'] = gdf_mode.loc[:, 'nhisp_car':'nhisp_means_other'].div(gdf_mode.loc[:, 'nhisp_car':'nhisp_means_other'].sum(axis=1), axis=0)
gdf_mode.to_csv("data/chicago/var/mode.csv", index=False)
gdf_mode

,GEOID_Data,car,pub_trans,means_other,white_car,white_pub_trans,white_means_other,black_car,black_pub_trans,black_means_other,...,other_means_other,nhisp_white_car,nhisp_white_pub_trans,nhisp_white_means_other,hisp_car,hisp_pub_trans,hisp_means_other,nhisp_car,nhisp_pub_trans,nhisp_means_other
0,14000US17031221000,0.580012,0.223789,0.196199,0.527473,0.238095,0.234432,0.333333,0.326087,0.340580,...,0.042394,0.406516,0.270538,0.322946,0.786501,0.158402,0.055096,0.414365,0.276243,0.309392
1,14000US17031221100,0.387560,0.326463,0.285977,0.362023,0.315761,0.322216,0.658228,0.341772,0.000000,...,0.254286,0.329863,0.303839,0.366298,0.569444,0.286458,0.144097,0.338627,0.337226,0.324148
2,14000US17031242800,0.492386,0.189848,0.317766,0.495614,0.197368,0.307018,0.500000,0.000000,0.500000,...,0.442623,0.482005,0.173522,0.344473,0.522222,0.288889,0.188889,0.485714,0.167702,0.346584
3,14000US17031242900,0.522117,0.162717,0.315166,0.523649,0.200450,0.275901,0.343284,0.268657,0.388060,...,0.411576,0.544471,0.185096,0.270433,0.544304,0.118143,0.337553,0.517007,0.172983,0.310010
4,14000US17031243000,0.517845,0.233730,0.248425,0.482531,0.257790,0.259679,0.000000,0.444444,0.555556,...,0.198864,0.471400,0.257396,0.271203,0.674419,0.174419,0.151163,0.496420,0.241846,0.261734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.478413,0.348888,0.172700,1.000000,0.000000,0.000000,0.488838,0.364465,0.146697,...,1.000000,1.000000,0.000000,0.000000,NaN,NaN,NaN,0.478413,0.348888,0.172700
793,14000US17031540101,0.619377,0.320069,0.060554,1.000000,0.000000,0.000000,0.611262,0.336058,0.052679,...,1.000000,NaN,NaN,NaN,0.781818,0.000000,0.218182,0.611262,0.336058,0.052679
794,14000US17031050300,0.539085,0.139548,0.321367,0.600000,0.152941,0.247059,0.375000,0.000000,0.625000,...,0.481766,0.601129,0.132643,0.266228,0.583333,0.200758,0.215909,0.531100,0.128503,0.340396
795,14000US17031830700,0.342535,0.399414,0.258051,0.271390,0.453209,0.275401,0.494118,0.262745,0.243137,...,0.210390,0.303354,0.434451,0.262195,0.227758,0.437722,0.334520,0.357820,0.394313,0.247867


Poverty

In [25]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B17001")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X17_POVERTY', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['below_pov'] = gdf['B17001e2']
gdf['above_pov'] = gdf['B17001e31']

## white
gdf['white_below_pov'] = gdf['B17001Ae2']
gdf['white_above_pov'] = gdf['B17001Ae31']

## black
gdf['black_below_pov'] = gdf['B17001Ae2']
gdf['black_above_pov'] = gdf['B17001Ae31']
gdf_pov = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not value.startswith("B17001")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_pov = gdf_pov.drop(columns=['GEOID', 'geometry'])

## other races
gdf_pov['other_below_pov'] = gdf_pov['below_pov'] - gdf_pov['white_below_pov'] - gdf_pov['black_below_pov']
gdf_pov['other_above_pov'] = gdf_pov['above_pov'] - gdf_pov['white_above_pov'] - gdf_pov['black_above_pov']

## white & non-hispanic
gdf['nhisp_white_below_pov'] = gdf['B17001He2']
gdf['nhisp_white_above_pov'] = gdf['B17001He31']

## hispanic
gdf['hisp_below_pov'] = gdf['B17001Ie2']
gdf['hisp_above_pov'] = gdf['B17001Ie31']
gdf_pov = gdf_pov.merge(gdf[[value for value in gdf.columns if value=="GEOID" or value.startswith("nhisp") or value.startswith("hisp")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_pov = gdf_pov.drop(columns=['GEOID'])

## non-hispanic
gdf_pov['nhisp_below_pov'] = gdf_pov['below_pov'] - gdf_pov['hisp_below_pov']
gdf_pov['nhisp_above_pov'] = gdf_pov['above_pov'] - gdf_pov['hisp_above_pov']

gdf_pov

,GEOID_Data,below_pov,above_pov,white_below_pov,white_above_pov,black_below_pov,black_above_pov,other_below_pov,other_above_pov,nhisp_white_below_pov,nhisp_white_above_pov,hisp_below_pov,hisp_above_pov,nhisp_below_pov,nhisp_above_pov
0,14000US17031221000,250.0,2320.0,126.0,1573.0,126.0,1573.0,-2.0,-826.0,88.0,951.0,130.0,1153.0,120.0,1167.0
1,14000US17031221100,753.0,3535.0,276.0,2498.0,276.0,2498.0,201.0,-1461.0,144.0,1799.0,496.0,1060.0,257.0,2475.0
2,14000US17031242800,93.0,1458.0,64.0,1302.0,64.0,1302.0,-35.0,-1146.0,45.0,1117.0,42.0,313.0,51.0,1145.0
3,14000US17031242900,77.0,1792.0,35.0,1208.0,35.0,1208.0,7.0,-624.0,15.0,1130.0,51.0,382.0,26.0,1410.0
4,14000US17031243000,83.0,2118.0,47.0,1335.0,47.0,1335.0,-11.0,-552.0,47.0,1272.0,0.0,535.0,83.0,1583.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,1872.0,4306.0,0.0,24.0,0.0,24.0,1872.0,4258.0,0.0,24.0,0.0,0.0,1872.0,4306.0
793,14000US17031540101,2430.0,2105.0,103.0,60.0,103.0,60.0,2224.0,1985.0,17.0,0.0,86.0,104.0,2344.0,2001.0
794,14000US17031050300,144.0,2611.0,144.0,1636.0,144.0,1636.0,-144.0,-661.0,90.0,1478.0,54.0,435.0,90.0,2176.0
795,14000US17031830700,1003.0,3061.0,319.0,1649.0,319.0,1649.0,365.0,-237.0,319.0,1300.0,70.0,455.0,933.0,2606.0


In [26]:
gdf_pov.loc[:, 'below_pov':'above_pov'] = gdf_pov.loc[:, 'below_pov':'above_pov'].div(gdf_pov.loc[:, 'below_pov':'above_pov'].sum(axis=1), axis=0)
gdf_pov.loc[:, 'white_below_pov':'white_above_pov'] = gdf_pov.loc[:, 'white_below_pov':'white_above_pov'].div(gdf_pov.loc[:, 'white_below_pov':'white_above_pov'].sum(axis=1), axis=0)
gdf_pov.loc[:, 'black_below_pov':'black_above_pov'] = gdf_pov.loc[:, 'black_below_pov':'black_above_pov'].div(gdf_pov.loc[:, 'black_below_pov':'black_above_pov'].sum(axis=1), axis=0)
gdf_pov.loc[:, 'other_below_pov':'other_above_pov'] = gdf_pov.loc[:, 'other_below_pov':'other_above_pov'].div(gdf_pov.loc[:, 'other_below_pov':'other_above_pov'].sum(axis=1), axis=0)
gdf_pov.loc[:, 'nhisp_white_below_pov':'nhisp_white_above_pov'] = gdf_pov.loc[:, 'nhisp_white_below_pov':'nhisp_white_above_pov'].div(gdf_pov.loc[:, 'nhisp_white_below_pov':'nhisp_white_above_pov'].sum(axis=1), axis=0)
gdf_pov.loc[:, 'hisp_below_pov':'hisp_above_pov'] = gdf_pov.loc[:, 'hisp_below_pov':'hisp_above_pov'].div(gdf_pov.loc[:, 'hisp_below_pov':'hisp_above_pov'].sum(axis=1), axis=0)
gdf_pov.loc[:, 'nhisp_below_pov':'nhisp_above_pov'] = gdf_pov.loc[:, 'nhisp_below_pov':'nhisp_above_pov'].div(gdf_pov.loc[:, 'nhisp_below_pov':'nhisp_above_pov'].sum(axis=1), axis=0)
gdf_pov.to_csv("data/chicago/var/poverty.csv", index=False)
gdf_pov

,GEOID_Data,below_pov,above_pov,white_below_pov,white_above_pov,black_below_pov,black_above_pov,other_below_pov,other_above_pov,nhisp_white_below_pov,nhisp_white_above_pov,hisp_below_pov,hisp_above_pov,nhisp_below_pov,nhisp_above_pov
0,14000US17031221000,0.097276,0.902724,0.074161,0.925839,0.074161,0.925839,0.002415,0.997585,0.084697,0.915303,0.101325,0.898675,0.093240,0.906760
1,14000US17031221100,0.175606,0.824394,0.099495,0.900505,0.099495,0.900505,-0.159524,1.159524,0.074112,0.925888,0.318766,0.681234,0.094070,0.905930
2,14000US17031242800,0.059961,0.940039,0.046852,0.953148,0.046852,0.953148,0.029636,0.970364,0.038726,0.961274,0.118310,0.881690,0.042642,0.957358
3,14000US17031242900,0.041199,0.958801,0.028158,0.971842,0.028158,0.971842,-0.011345,1.011345,0.013100,0.986900,0.117783,0.882217,0.018106,0.981894
4,14000US17031243000,0.037710,0.962290,0.034009,0.965991,0.034009,0.965991,0.019538,0.980462,0.035633,0.964367,0.000000,1.000000,0.049820,0.950180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.303011,0.696989,0.000000,1.000000,0.000000,1.000000,0.305383,0.694617,0.000000,1.000000,NaN,NaN,0.303011,0.696989
793,14000US17031540101,0.535832,0.464168,0.631902,0.368098,0.631902,0.368098,0.528392,0.471608,1.000000,0.000000,0.452632,0.547368,0.539471,0.460529
794,14000US17031050300,0.052269,0.947731,0.080899,0.919101,0.080899,0.919101,0.178882,0.821118,0.057398,0.942602,0.110429,0.889571,0.039718,0.960282
795,14000US17031830700,0.246801,0.753199,0.162093,0.837907,0.162093,0.837907,2.851562,-1.851562,0.197035,0.802965,0.133333,0.866667,0.263634,0.736366


Household income

In [28]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B19001")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X19_INCOME', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['below_45000'] = gdf[[f'B19001e{i}' for i in range(2, 10)]].sum(axis=1)
gdf['45000_to_100000'] = gdf[[f'B19001e{i}' for i in range(10, 14)]].sum(axis=1)
gdf['above_100000'] = gdf[[f'B19001e{i}' for i in range(14, 18)]].sum(axis=1)

## white
gdf['white_below_45000'] = gdf[[f'B19001Ae{i}' for i in range(2, 10)]].sum(axis=1)
gdf['white_45000_to_100000'] = gdf[[f'B19001Ae{i}' for i in range(10, 14)]].sum(axis=1)
gdf['white_above_100000'] = gdf[[f'B19001Ae{i}' for i in range(14, 18)]].sum(axis=1)

## black
gdf['black_below_45000'] = gdf[[f'B19001Be{i}' for i in range(2, 10)]].sum(axis=1)
gdf['black_45000_to_100000'] = gdf[[f'B19001Be{i}' for i in range(10, 14)]].sum(axis=1)
gdf['black_above_100000'] = gdf[[f'B19001Be{i}' for i in range(14, 18)]].sum(axis=1)
gdf_hhinc = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not value.startswith("B19001")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_hhinc = gdf_hhinc.drop(columns=['GEOID', 'geometry'])

## other races
gdf_hhinc['other_below_45000'] = gdf_hhinc['below_45000'] - gdf_hhinc['white_below_45000'] - gdf_hhinc['black_below_45000']
gdf_hhinc['other_45000_to_100000'] = gdf_hhinc['45000_to_100000'] - gdf_hhinc['white_45000_to_100000'] - gdf_hhinc['black_45000_to_100000']
gdf_hhinc['other_above_100000'] = gdf_hhinc['above_100000'] - gdf_hhinc['white_above_100000'] - gdf_hhinc['black_above_100000']

## white & non-hispanic
gdf['nhisp_white_below_45000'] = gdf[[f'B19001He{i}' for i in range(2, 10)]].sum(axis=1)
gdf['nhisp_white_45000_to_100000'] = gdf[[f'B19001He{i}' for i in range(10, 14)]].sum(axis=1)
gdf['nhisp_white_above_100000'] = gdf[[f'B19001He{i}' for i in range(14, 18)]].sum(axis=1)

## hispanic
gdf['hisp_below_45000'] = gdf[[f'B19001Ie{i}' for i in range(2, 10)]].sum(axis=1)
gdf['hisp_45000_to_100000'] = gdf[[f'B19001Ie{i}' for i in range(10, 14)]].sum(axis=1)
gdf['hisp_above_100000'] = gdf[[f'B19001Ie{i}' for i in range(14, 18)]].sum(axis=1)
gdf_hhinc = gdf_hhinc.merge(gdf[[value for value in gdf.columns if value=="GEOID" or value.startswith("nhisp") or value.startswith("hisp")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_hhinc = gdf_hhinc.drop(columns=['GEOID'])

## non-hispanic
gdf_hhinc['nhisp_below_45000'] = gdf_hhinc['below_45000'] - gdf_hhinc['hisp_below_45000']
gdf_hhinc['nhisp_45000_to_100000'] = gdf_hhinc['45000_to_100000'] - gdf_hhinc['hisp_45000_to_100000']
gdf_hhinc['nhisp_above_100000'] = gdf_hhinc['above_100000'] - gdf_hhinc['hisp_above_100000']

gdf_hhinc

,GEOID_Data,below_45000,45000_to_100000,above_100000,white_below_45000,white_45000_to_100000,white_above_100000,black_below_45000,black_45000_to_100000,black_above_100000,...,other_above_100000,nhisp_white_below_45000,nhisp_white_45000_to_100000,nhisp_white_above_100000,hisp_below_45000,hisp_45000_to_100000,hisp_above_100000,nhisp_below_45000,nhisp_45000_to_100000,nhisp_above_100000
0,14000US17031221000,302.0,374.0,387.0,224.0,249.0,244.0,17.0,18.0,30.0,...,113.0,122.0,154.0,220.0,163.0,182.0,121.0,139.0,192.0,266.0
1,14000US17031221100,606.0,652.0,753.0,321.0,495.0,589.0,94.0,12.0,25.0,...,139.0,86.0,447.0,519.0,420.0,92.0,110.0,186.0,560.0,643.0
2,14000US17031242800,84.0,210.0,366.0,79.0,198.0,339.0,0.0,0.0,0.0,...,27.0,74.0,165.0,309.0,10.0,40.0,47.0,74.0,170.0,319.0
3,14000US17031242900,111.0,235.0,456.0,73.0,129.0,379.0,0.0,54.0,5.0,...,72.0,37.0,129.0,373.0,57.0,45.0,29.0,54.0,190.0,427.0
4,14000US17031243000,106.0,414.0,559.0,91.0,343.0,338.0,0.0,0.0,10.0,...,211.0,91.0,327.0,338.0,15.0,48.0,116.0,91.0,366.0,443.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,1499.0,699.0,297.0,0.0,0.0,0.0,1453.0,625.0,297.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1499.0,699.0,297.0
793,14000US17031540101,1350.0,142.0,169.0,31.0,0.0,15.0,1304.0,142.0,142.0,...,12.0,0.0,0.0,0.0,31.0,0.0,27.0,1319.0,142.0,142.0
794,14000US17031050300,181.0,239.0,671.0,170.0,190.0,475.0,0.0,10.0,0.0,...,196.0,87.0,181.0,444.0,83.0,32.0,82.0,98.0,207.0,589.0
795,14000US17031830700,784.0,477.0,658.0,410.0,265.0,435.0,242.0,159.0,51.0,...,172.0,314.0,254.0,410.0,131.0,37.0,32.0,653.0,440.0,626.0


In [29]:
gdf_hhinc.loc[:, 'below_45000':'above_100000'] = gdf_hhinc.loc[:, 'below_45000':'above_100000'].div(gdf_hhinc.loc[:, 'below_45000':'above_100000'].sum(axis=1), axis=0)
gdf_hhinc.loc[:, 'white_below_45000':'white_above_100000'] = gdf_hhinc.loc[:, 'white_below_45000':'white_above_100000'].div(gdf_hhinc.loc[:, 'white_below_45000':'white_above_100000'].sum(axis=1), axis=0)
gdf_hhinc.loc[:, 'black_below_45000':'black_above_100000'] = gdf_hhinc.loc[:, 'black_below_45000':'black_above_100000'].div(gdf_hhinc.loc[:, 'black_below_45000':'black_above_100000'].sum(axis=1), axis=0)
gdf_hhinc.loc[:, 'other_below_45000':'other_above_100000'] = gdf_hhinc.loc[:, 'other_below_45000':'other_above_100000'].div(gdf_hhinc.loc[:, 'other_below_45000':'other_above_100000'].sum(axis=1), axis=0)
gdf_hhinc.loc[:, 'nhisp_white_below_45000':'nhisp_white_above_100000'] = gdf_hhinc.loc[:, 'nhisp_white_below_45000':'nhisp_white_above_100000'].div(gdf_hhinc.loc[:, 'nhisp_white_below_45000':'nhisp_white_above_100000'].sum(axis=1), axis=0)
gdf_hhinc.loc[:, 'hisp_below_45000':'hisp_above_100000'] = gdf_hhinc.loc[:, 'hisp_below_45000':'hisp_above_100000'].div(gdf_hhinc.loc[:, 'hisp_below_45000':'hisp_above_100000'].sum(axis=1), axis=0)
gdf_hhinc.loc[:, 'nhisp_below_45000':'nhisp_above_100000'] = gdf_hhinc.loc[:, 'nhisp_below_45000':'nhisp_above_100000'].div(gdf_hhinc.loc[:, 'nhisp_below_45000':'nhisp_above_100000'].sum(axis=1), axis=0)
gdf_hhinc.to_csv("data/chicago/var/household_income.csv", index=False)
gdf_hhinc

,GEOID_Data,below_45000,45000_to_100000,above_100000,white_below_45000,white_45000_to_100000,white_above_100000,black_below_45000,black_45000_to_100000,black_above_100000,...,other_above_100000,nhisp_white_below_45000,nhisp_white_45000_to_100000,nhisp_white_above_100000,hisp_below_45000,hisp_45000_to_100000,hisp_above_100000,nhisp_below_45000,nhisp_45000_to_100000,nhisp_above_100000
0,14000US17031221000,0.284102,0.351834,0.364064,0.312413,0.347280,0.340307,0.261538,0.276923,0.461538,...,0.402135,0.245968,0.310484,0.443548,0.349785,0.390558,0.259657,0.232831,0.321608,0.445561
1,14000US17031221100,0.301343,0.324217,0.374441,0.228470,0.352313,0.419217,0.717557,0.091603,0.190840,...,0.292632,0.081749,0.424905,0.493346,0.675241,0.147910,0.176849,0.133909,0.403168,0.462923
2,14000US17031242800,0.127273,0.318182,0.554545,0.128247,0.321429,0.550325,NaN,NaN,NaN,...,0.613636,0.135036,0.301095,0.563869,0.103093,0.412371,0.484536,0.131439,0.301954,0.566607
3,14000US17031242900,0.138404,0.293017,0.568579,0.125645,0.222031,0.652324,0.000000,0.915254,0.084746,...,0.444444,0.068646,0.239332,0.692022,0.435115,0.343511,0.221374,0.080477,0.283159,0.636364
4,14000US17031243000,0.098239,0.383689,0.518072,0.117876,0.444301,0.437824,0.000000,0.000000,1.000000,...,0.710438,0.120370,0.432540,0.447090,0.083799,0.268156,0.648045,0.101111,0.406667,0.492222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.600802,0.280160,0.119038,NaN,NaN,NaN,0.611789,0.263158,0.125053,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.600802,0.280160,0.119038
793,14000US17031540101,0.812763,0.085491,0.101746,0.673913,0.000000,0.326087,0.821159,0.089421,0.089421,...,0.444444,NaN,NaN,NaN,0.534483,0.000000,0.465517,0.822832,0.088584,0.088584
794,14000US17031050300,0.165903,0.219065,0.615032,0.203593,0.227545,0.568862,0.000000,1.000000,0.000000,...,0.796748,0.122191,0.254213,0.623596,0.421320,0.162437,0.416244,0.109620,0.231544,0.658837
795,14000US17031830700,0.408546,0.248567,0.342887,0.369369,0.238739,0.391892,0.535398,0.351770,0.112832,...,0.481793,0.321063,0.259714,0.419223,0.655000,0.185000,0.160000,0.379872,0.255963,0.364165


Occupation

In [32]:
## all
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("C24010")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X24_INDUSTRY_OCCUPATION', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['mgt'] = gdf[['C24010Ae3', 'C24010Ae9', 'C24010Be3', 'C24010Be9', 'C24010Ce3', 'C24010Ce9', 'C24010De3', 'C24010De9', 'C24010Ee3', 'C24010Ee9', 'C24010Fe3', 'C24010Fe9', 'C24010Ge3', 'C24010Ge9']].sum(axis=1)
gdf['svc'] = gdf[['C24010Ae4', 'C24010Ae10', 'C24010Be4', 'C24010Be10', 'C24010Ce4', 'C24010Ce10', 'C24010De4', 'C24010De10', 'C24010Ee4', 'C24010Ee10', 'C24010Fe4', 'C24010Fe10', 'C24010Ge4', 'C24010Ge10']].sum(axis=1)
gdf['sls'] = gdf[['C24010Ae5', 'C24010Ae11', 'C24010Be5', 'C24010Be11', 'C24010Ce5', 'C24010Ce11', 'C24010De5', 'C24010De11', 'C24010Ee5', 'C24010Ee11', 'C24010Fe5', 'C24010Fe11', 'C24010Ge5', 'C24010Ge11']].sum(axis=1)
gdf['con'] = gdf[['C24010Ae6', 'C24010Ae12', 'C24010Be6', 'C24010Be12', 'C24010Ce6', 'C24010Ce12', 'C24010De6', 'C24010De12', 'C24010Ee6', 'C24010Ee12', 'C24010Fe6', 'C24010Fe12', 'C24010Ge6', 'C24010Ge12']].sum(axis=1)
gdf['prod'] = gdf[['C24010Ae7', 'C24010Ae13', 'C24010Be7', 'C24010Be13', 'C24010Ce7', 'C24010Ce13', 'C24010De7', 'C24010De13', 'C24010Ee7', 'C24010Ee13', 'C24010Fe7', 'C24010Fe13', 'C24010Ge7', 'C24010Ge13']].sum(axis=1)

## white
gdf['white_mgt'] = gdf['C24010Ae3'] + gdf['C24010Ae9']
gdf['white_svc'] = gdf['C24010Ae4'] + gdf['C24010Ae10']
gdf['white_sls'] = gdf['C24010Ae5'] + gdf['C24010Ae11']
gdf['white_con'] = gdf['C24010Ae6'] + gdf['C24010Ae12']
gdf['white_prod'] = gdf['C24010Ae7'] + gdf['C24010Ae13']

## black
gdf['black_mgt'] = gdf['C24010Be3'] + gdf['C24010Be9']
gdf['black_svc'] = gdf['C24010Be4'] + gdf['C24010Be10']
gdf['black_sls'] = gdf['C24010Be5'] + gdf['C24010Be11']
gdf['black_con'] = gdf['C24010Be6'] + gdf['C24010Be12']
gdf['black_prod'] = gdf['C24010Be7'] + gdf['C24010Be13']

## other races
gdf['other_mgt'] = gdf[['C24010Ce3', 'C24010Ce9', 'C24010De3', 'C24010De9', 'C24010Ee3', 'C24010Ee9', 'C24010Fe3', 'C24010Fe9', 'C24010Ge3', 'C24010Ge9']].sum(axis=1)
gdf['other_svc'] = gdf[['C24010Ce4', 'C24010Ce10', 'C24010De4', 'C24010De10', 'C24010Ee4', 'C24010Ee10', 'C24010Fe4', 'C24010Fe10', 'C24010Ge4', 'C24010Ge10']].sum(axis=1)
gdf['other_sls'] = gdf[['C24010Ce5', 'C24010Ce11', 'C24010De5', 'C24010De11', 'C24010Ee5', 'C24010Ee11', 'C24010Fe5', 'C24010Fe11', 'C24010Ge5', 'C24010Ge11']].sum(axis=1)
gdf['other_con'] = gdf[['C24010Ce6', 'C24010Ce12', 'C24010De6', 'C24010De12', 'C24010Ee6', 'C24010Ee12', 'C24010Fe6', 'C24010Fe12', 'C24010Ge6', 'C24010Ge12']].sum(axis=1)
gdf['other_prod'] = gdf[['C24010Ce7', 'C24010Ce13', 'C24010De7', 'C24010De13', 'C24010Ee7', 'C24010Ee13', 'C24010Fe7', 'C24010Fe13', 'C24010Ge7', 'C24010Ge13']].sum(axis=1)

## white & non-hispanic
gdf['nhisp_white_mgt'] = gdf['C24010He3'] + gdf['C24010He9']
gdf['nhisp_white_svc'] = gdf['C24010He4'] + gdf['C24010He10']
gdf['nhisp_white_sls'] = gdf['C24010He5'] + gdf['C24010He11']
gdf['nhisp_white_con'] = gdf['C24010He6'] + gdf['C24010He12']
gdf['nhisp_white_prod'] = gdf['C24010He7'] + gdf['C24010He13']

## hispanic
gdf['hisp_mgt'] = gdf['C24010Ie3'] + gdf['C24010Ie9']
gdf['hisp_svc'] = gdf['C24010Ie4'] + gdf['C24010Ie10']
gdf['hisp_sls'] = gdf['C24010Ie5'] + gdf['C24010Ie11']
gdf['hisp_con'] = gdf['C24010Ie6'] + gdf['C24010Ie12']
gdf['hisp_prod'] = gdf['C24010Ie7'] + gdf['C24010Ie13']
gdf_occ = gdf_geo[['GEOID_Data']].merge(gdf[[value for value in gdf.columns if not value.startswith("C24010")]], left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_occ = gdf_occ.drop(columns=['GEOID', 'geometry'])

## non-hispanic
gdf_occ['nhisp_mgt'] = gdf_occ['mgt'] - gdf_occ['hisp_mgt']
gdf_occ['nhisp_svc'] = gdf_occ['svc'] - gdf_occ['hisp_svc']
gdf_occ['nhisp_sls'] = gdf_occ['sls'] - gdf_occ['hisp_sls']
gdf_occ['nhisp_con'] = gdf_occ['con'] - gdf_occ['hisp_con']
gdf_occ['nhisp_prod'] = gdf_occ['prod'] - gdf_occ['hisp_prod']

gdf_occ

,GEOID_Data,mgt,svc,sls,con,prod,white_mgt,white_svc,white_sls,white_con,...,hisp_mgt,hisp_svc,hisp_sls,hisp_con,hisp_prod,nhisp_mgt,nhisp_svc,nhisp_sls,nhisp_con,nhisp_prod
0,14000US17031221000,757.0,313.0,316.0,139.0,128.0,516.0,151.0,254.0,101.0,...,163.0,181.0,209.0,84.0,89.0,594.0,132.0,107.0,55.0,39.0
1,14000US17031221100,1715.0,405.0,417.0,51.0,161.0,1284.0,198.0,245.0,28.0,...,149.0,193.0,82.0,41.0,134.0,1566.0,212.0,335.0,10.0,27.0
2,14000US17031242800,592.0,134.0,130.0,56.0,74.0,577.0,116.0,113.0,45.0,...,33.0,63.0,27.0,51.0,6.0,559.0,71.0,103.0,5.0,68.0
3,14000US17031242900,881.0,130.0,168.0,48.0,44.0,633.0,63.0,117.0,48.0,...,96.0,77.0,52.0,0.0,12.0,785.0,53.0,116.0,48.0,32.0
4,14000US17031243000,951.0,183.0,267.0,51.0,34.0,730.0,123.0,196.0,19.0,...,71.0,48.0,39.0,23.0,0.0,880.0,135.0,228.0,28.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,671.0,817.0,604.0,34.0,148.0,0.0,24.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,671.0,817.0,604.0,34.0,148.0
793,14000US17031540101,271.0,368.0,320.0,25.0,226.0,43.0,0.0,0.0,0.0,...,55.0,0.0,0.0,0.0,0.0,216.0,368.0,320.0,25.0,226.0
794,14000US17031050300,1095.0,182.0,336.0,32.0,102.0,770.0,135.0,219.0,20.0,...,137.0,55.0,69.0,12.0,0.0,958.0,127.0,267.0,20.0,102.0
795,14000US17031830700,1379.0,285.0,413.0,66.0,280.0,984.0,119.0,261.0,29.0,...,92.0,88.0,70.0,19.0,12.0,1287.0,197.0,343.0,47.0,268.0


In [33]:
gdf_occ.loc[:, 'mgt':'prod'] = gdf_occ.loc[:, 'mgt':'prod'].div(gdf_occ.loc[:, 'mgt':'prod'].sum(axis=1), axis=0)
gdf_occ.loc[:, 'white_mgt':'white_prod'] = gdf_occ.loc[:, 'white_mgt':'white_prod'].div(gdf_occ.loc[:, 'white_mgt':'white_prod'].sum(axis=1), axis=0)
gdf_occ.loc[:, 'black_mgt':'black_prod'] = gdf_occ.loc[:, 'black_mgt':'black_prod'].div(gdf_occ.loc[:, 'black_mgt':'black_prod'].sum(axis=1), axis=0)
gdf_occ.loc[:, 'other_mgt':'other_prod'] = gdf_occ.loc[:, 'other_mgt':'other_prod'].div(gdf_occ.loc[:, 'other_mgt':'other_prod'].sum(axis=1), axis=0)
gdf_occ.loc[:, 'nhisp_white_mgt':'nhisp_white_prod'] = gdf_occ.loc[:, 'nhisp_white_mgt':'nhisp_white_prod'].div(gdf_occ.loc[:, 'nhisp_white_mgt':'nhisp_white_prod'].sum(axis=1), axis=0)
gdf_occ.loc[:, 'hisp_mgt':'hisp_prod'] = gdf_occ.loc[:, 'hisp_mgt':'hisp_prod'].div(gdf_occ.loc[:, 'hisp_mgt':'hisp_prod'].sum(axis=1), axis=0)
gdf_occ.loc[:, 'nhisp_mgt':'nhisp_prod'] = gdf_occ.loc[:, 'nhisp_mgt':'nhisp_prod'].div(gdf_occ.loc[:, 'nhisp_mgt':'nhisp_prod'].sum(axis=1), axis=0)
gdf_occ.to_csv("data/chicago/var/occupation.csv", index=False)
gdf_occ

,GEOID_Data,mgt,svc,sls,con,prod,white_mgt,white_svc,white_sls,white_con,...,hisp_mgt,hisp_svc,hisp_sls,hisp_con,hisp_prod,nhisp_mgt,nhisp_svc,nhisp_sls,nhisp_con,nhisp_prod
0,14000US17031221000,0.457955,0.189353,0.191168,0.084090,0.077435,0.463196,0.135548,0.228007,0.090664,...,0.224518,0.249311,0.287879,0.115702,0.122590,0.640777,0.142395,0.115426,0.059331,0.042071
1,14000US17031221100,0.623863,0.147326,0.151692,0.018552,0.058567,0.682253,0.105207,0.130181,0.014878,...,0.248748,0.322204,0.136895,0.068447,0.223706,0.728372,0.098605,0.155814,0.004651,0.012558
2,14000US17031242800,0.600406,0.135903,0.131846,0.056795,0.075051,0.631982,0.127054,0.123768,0.049288,...,0.183333,0.350000,0.150000,0.283333,0.033333,0.693548,0.088089,0.127792,0.006203,0.084367
3,14000US17031242900,0.693155,0.102282,0.132179,0.037766,0.034618,0.708847,0.070549,0.131019,0.053751,...,0.405063,0.324895,0.219409,0.000000,0.050633,0.759188,0.051257,0.112186,0.046422,0.030948
4,14000US17031243000,0.639973,0.123149,0.179677,0.034320,0.022880,0.672192,0.113260,0.180479,0.017495,...,0.392265,0.265193,0.215470,0.127072,0.000000,0.674330,0.103448,0.174713,0.021456,0.026054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,14000US17031440201,0.295075,0.359279,0.265611,0.014952,0.065084,0.000000,1.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.295075,0.359279,0.265611,0.014952,0.065084
793,14000US17031540101,0.223967,0.304132,0.264463,0.020661,0.186777,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.187013,0.318615,0.277056,0.021645,0.195671
794,14000US17031050300,0.626789,0.104179,0.192330,0.018317,0.058386,0.636364,0.111570,0.180992,0.016529,...,0.501832,0.201465,0.252747,0.043956,0.000000,0.649932,0.086160,0.181140,0.013569,0.069199
795,14000US17031830700,0.569129,0.117623,0.170450,0.027239,0.115559,0.657754,0.079545,0.174465,0.019385,...,0.327402,0.313167,0.249110,0.067616,0.042705,0.600840,0.091970,0.160131,0.021942,0.125117


## Reorganize variables

In [5]:
import geopandas as gpd
import pandas as pd
import os

gdf_geo = gpd.read_file('data/chicago/chicago_ACS_2021_5YR_TRACT.shp')
gdf_geo = gdf_geo.rename(columns={'GEOID': 'GEOID_x'})

df_all = gdf_geo[['GEOID_Data', 'geometry']]
folder_path = 'data/chicago/var'
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        df = pd.read_csv(file_path)
        df_all = df_all.merge(df, left_on='GEOID_Data', right_on='GEOID_Data', how='left')

## white
cols = [col for col in df_all.columns if col.startswith('white_')]
df_white = df_all[['GEOID_Data', 'geometry'] + cols]
df_all.drop(columns=cols, inplace=True)
df_white.to_file('data/chicago/data_white.gpkg', driver='GPKG')

## black
cols = [col for col in df_all.columns if col.startswith('black_')]
df_black = df_all[['GEOID_Data', 'geometry'] + cols]
df_all.drop(columns=cols, inplace=True)
df_black.to_file('data/chicago/data_black.gpkg', driver='GPKG')

## other
cols = [col for col in df_all.columns if col.startswith('other_')]
df_other = df_all[['GEOID_Data', 'geometry'] + cols]
df_all.drop(columns=cols, inplace=True)
df_other.to_file('data/chicago/data_other.gpkg', driver='GPKG')

## white, nonhispanic
cols = [col for col in df_all.columns if col.startswith('nhisp_white_')]
df_nhisp_white = df_all[['GEOID_Data', 'geometry'] + cols]
df_all.drop(columns=cols, inplace=True)
df_nhisp_white.to_file('data/chicago/data_nhisp_white.gpkg', driver='GPKG')

## hispanic
cols = [col for col in df_all.columns if col.startswith('hisp_')]
df_hisp = df_all[['GEOID_Data', 'geometry'] + cols]
df_all.drop(columns=cols, inplace=True)
df_hisp.to_file('data/chicago/data_hisp.gpkg', driver='GPKG')

## non-hispanic
cols = [col for col in df_all.columns if col.startswith('nhisp_')]
df_nhisp = df_all[['GEOID_Data', 'geometry'] + cols]
df_all.drop(columns=cols, inplace=True)
df_nhisp.to_file('data/chicago/data_nhisp.gpkg', driver='GPKG')

## all
df_all.to_file("data/chicago/data_all.gpkg", driver='GPKG')
df_all.columns

Index(['GEOID_Data', 'geometry', 'ag1', 'ag2', 'ag3', 'ag4', 'ag5',
       'less_high_school', 'high_school', 'college', 'bachelor_higher',
       'below_45000', '45000_to_100000', 'above_100000',
       'married_couple_family', 'family_other', 'nonfamily', 'same_house',
       'moved_same_county', 'moved_same_state', 'moved_same_country',
       'moved_from_abroad', 'car', 'pub_trans', 'means_other', 'one_or_less',
       'more_than_one', 'mgt', 'svc', 'sls', 'con', 'prod', 'below_pov',
       'above_pov', 'white', 'black', 'race_other', 'hisp', 'nhisp', 'owner',
       'renter', 'single_family', 'multi_family', 'units_other'],
      dtype='object')

In [ ]:
import pandas as pd
import geopandas as gpd

meta = pd.read_csv("data/chicago/chicago_var.csv")
gdf_geo = gpd.read_file('data/chicago/chicago_ACS_2021_5YR_TRACT.shp')
gdf_geo = gdf_geo.rename(columns={'GEOID': 'GEOID_x'})

## race
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B02001e")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X02_RACE', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['total'] = gdf['B02001e1']
gdf['white'] = gdf['B02001e2']
gdf['other'] = gdf['B02001e3']
gdf['race_other'] = gdf['B02001e1'] - gdf['B02001e2'] - gdf['B02001e3']
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B02001e")])
gdf_race = gdf_geo[['GEOID_Data']].merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_race = gdf_race.drop(columns=['GEOID', 'geometry'])

## hispanic origin
cols_sel = ['GEOID'] + [value for value in meta['Short_Name'] if value.startswith("B03001e")]
gdf = gpd.read_file('data/chicago/ACS_2021_5YR_TRACT.gdb/ACS_2021_5YR_TRACT.gdb', driver='FileGDB', layer='X03_HISPANIC_OR_LATINO_ORIGIN', include_fields=cols_sel)
gdf = gdf[gdf['GEOID'].isin(gdf_geo['GEOID_Data'])]
gdf['hisp'] = gdf['B03001e3']
gdf['nhisp'] = gdf['B03001e2']
gdf = gdf.drop(columns=[value for value in meta['Short_Name'] if value.startswith("B03001e")])
gdf_race = gdf_race.merge(gdf, left_on='GEOID_Data', right_on='GEOID', how='left')
gdf_race = gdf_race.drop(columns=['GEOID', 'geometry'])

df_all = gdf_geo[['GEOID_Data', 'geometry']].merge(gdf_race, left_on='GEOID_Data', right_on='GEOID_Data', how='left')
df_all.to_file('data/chicago/data_pop.gpkg', driver='GPKG')